# Analyze Unlabel from Re-label 3

1. Import data from Relabel_result_3.xlsx
2. Select data for each observation that are unlabeled
3. Find patterns using RegEx and N-grams (use function from Analyze_group_c.ipynb)
4. Find word before and after keyword
5. Add the result to negative, positive and uncertain key in Relabel_group_c.ipynb
6. Re-label again in Relabel_group_c.ipynb

## Import Data

In [139]:
# from google.colab import drive

# drive.mount('/content/drive')

In [140]:
# %cd /content/drive/My Drive/Pre-WiL

In [141]:
import pandas as pd

cardio_data_old = pd.read_excel('Relabel_result_2.xlsx', sheet_name='Cardiomegaly')
pleural_data_old = pd.read_excel('Relabel_result_2.xlsx', sheet_name='Pleural Effusion')
atelec_data_old = pd.read_excel('Relabel_result_2.xlsx', sheet_name='Atelectasis')
lesion_data_old = pd.read_excel('Relabel_result_2.xlsx', sheet_name='Lung Lesion')
v1_data_old = pd.read_excel('Relabel_result_2.xlsx', sheet_name='Inspectra Lung Opacity v1')
edema_data_old = pd.read_excel('Relabel_result_2.xlsx', sheet_name='Edema')

In [142]:
cardio_data = pd.read_excel('Relabel_result_sym.xlsx', sheet_name='Cardiomegaly')
pleural_data = pd.read_excel('Relabel_result_sym.xlsx', sheet_name='Pleural Effusion')
atelec_data = pd.read_excel('Relabel_result_sym.xlsx', sheet_name='Atelectasis')
lesion_data = pd.read_excel('Relabel_result_sym.xlsx', sheet_name='Lung Lesion')
v1_data = pd.read_excel('Relabel_result_sym.xlsx', sheet_name='Inspectra Lung Opacity v1')
# edema_data = pd.read_excel('Relabel_result_sym.xlsx', sheet_name='Edema')

## Function

In [143]:
# create dataframe of each observations
def selectData(df, name, value):
  # choose rows
  if value == None:
    name_df = df[df[name + ' Ground Truth'].isna()]
  else:
    name_df = df[df[name + ' Ground Truth'] == value]

  return name_df

In [144]:
# create list of reports
import re
from ast import literal_eval

def createReportList(df):
  return list(map(literal_eval, df['Report List']))

In [145]:
# create n-grams
from collections import defaultdict
import re
from nltk import ngrams

def createPatternNgrams(data, n, keyword=None):
  pattern_dict = defaultdict(lambda: 0)
  for report in data:
    for sent in report[1:]:
      if (keyword != None and re.search(keyword, sent)) or keyword == None:
        for word in ngrams(sent.split(), n):
          pattern_dict[word] += 1    

  pattern_dict = sorted(pattern_dict.items(), key=lambda x: x[1], reverse=True)

  return dict(pattern_dict)

In [146]:
# find word before input word
def findWordBefore(bigrams_dict, word):
  word_before = defaultdict(lambda: 0)
  for k,v in bigrams_dict.items():
    if re.search(word, k[1]):
      word_before[k[0]] += v

  word_before = sorted(word_before.items(), key=lambda x: x[1], reverse=True)

  return dict(word_before)

def findWordBefore2(trigrams_dict, word):
  word_before = defaultdict(lambda: 0)
  for k,v in trigrams_dict.items():
    if re.search(word, k[2]):
      word_before[k[0]] += v

  word_before = sorted(word_before.items(), key=lambda x: x[1], reverse=True)

  return dict(word_before)

In [147]:
# find word after v.to be
def findWordAfter(trigrams_dict, word):
  word_after = defaultdict(lambda: 0)
  for k,v in trigrams_dict.items():
    if re.search(word, k[0]) and re.search(r'is|are|was|were', k[1]):
      word_after[k[2]] += v

  word_after = sorted(word_after.items(), key=lambda x: x[1], reverse=True)

  return dict(word_after)

In [148]:
def findDiffData(name, oldDf, df):
    for i in df.index:
        if not (pd.isna(df.at[i, name+' Ground Truth']) and  pd.isna(oldDf.at[i, name+' Ground Truth'])):
            if df.at[i, name+' Ground Truth'] != oldDf.at[i, name+' Ground Truth']:
                print(oldDf.at[i, name+' Ground Truth'], df.at[i, name+' Ground Truth'], df.at[i,'Report List'])

In [149]:
def findDiffData2(name, oldDf, df):
    for i in df.index:
        if not (pd.isna(df.at[i, name+' Ground Truth']) and  pd.isna(oldDf.at[i, name+' Ground Truth'])):
            if df.at[i, name+' Ground Truth'] != oldDf.at[i, name+' Ground Truth']:
                print(oldDf.at[i, name+' Ground Truth'], df.at[i, name+' Ground Truth'], oldDf.at[i,'Report List'])
                print('         ' + df.at[i,'Report List'])

# Cardiomegaly

In [150]:
# define keyword
cardio_keyword = r'card|heart|lv|ventricular enlarge'

## Ground truth 0

Prepare Data

In [151]:
# choose reports
cardio_df_0 = selectData(cardio_data, 'Cardiomegaly', 0.0)
cardio_df_0

,Reports,Report List,Cardiomegaly BERT Labeler,Cardiomegaly Inspectra Labeler,Cardiomegaly Ground Truth,Cardiomegaly Key Detect
1,CXR PA upright\n The chest shows round calcif...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,0.0,"<re.Match object; span=(8, 21), match='heart i..."
10,CHEST\n\nMinimal fibrotic change at LUL.\nThe ...,"['<s> chest <\\s>', '<s> minimal fibrotic chan...",0,1,0.0,0.5
14,CHEST.;\nLarge soft tissue mass at right parat...,"['<s> chest <\\s>', '<s> large soft tissue mas...",1,0,0.0,"<re.Match object; span=(4, 53), match='no card..."
16,CHEST PA UPRIGHT\n\nhistory of ca lung post ri...,"['<s> chest pa upright <\\s>', '<s> history of...",1,0,0.0,"<re.Match object; span=(4, 29), match='no card..."
19,CXR \n\nHistory of hemoptysis\n\nThere is mass...,"['<s> cxr <\\s>', '<s> history of hemoptysis <...",1,0,0.0,"<re.Match object; span=(13, 33), match='no car..."
...,...,...,...,...,...,...
39236,CXR PA UPRIGHT\n\nFINDINGS : No detectable pu...,"['<s> cxr pa upright <\\s>', '<s> findings no ...",1,0,0.0,"<re.Match object; span=(4, 32), match='no card..."
39237,CXR PA UPRIGHT\n\nFINDINGS : No detectable pu...,"['<s> cxr pa upright <\\s>', '<s> findings no ...",1,0,0.0,"<re.Match object; span=(4, 32), match='no card..."
39238,CXR(PA)\n\n IMP:\n No active pulmonary infi...,"['<s> cxr pa <\\s>', '<s> imp <\\s>', '<s> no ...",1,0,0.0,"<re.Match object; span=(13, 33), match='no car..."
39239,CXR(PA)\n\n IMP:\n No active pulmonary infi...,"['<s> cxr pa <\\s>', '<s> imp <\\s>', '<s> no ...",1,0,0.0,"<re.Match object; span=(13, 33), match='no car..."


In [152]:
# create report list
cardio_report_0 = createReportList(cardio_df_0)
cardio_report_0[0]

['<s> cxr pa upright <\\s>',
 '<s> the chest shows round calcification at cardiac shadow should be calcification at left lower lung <\\s>',
 '<s> the right lung is clear <\\s>',
 '<s> the heart is not enlarged <\\s>',
 '<s> both costophrenic are clear <\\s>',
 '<s> the bony thorax is intact <\\s>']

Find N-grams Pattern

In [153]:
cardio_bigram_0 = createPatternNgrams(cardio_report_0, 2, cardio_keyword)
list(cardio_bigram_0.items())[:20]

[(('no', 'cardiomegaly'), 30077),
 (('<s>', 'no'), 29859),
 (('cardiomegaly', 'is'), 27472),
 (('is', 'observed'), 10772),
 (('observed', '<\\s>'), 10766),
 (('seen', '<\\s>'), 9987),
 (('is', 'seen'), 9958),
 (('noted', '<\\s>'), 3449),
 (('is', 'noted'), 3402),
 (('found', '<\\s>'), 3091),
 (('is', 'found'), 3089),
 (('cardiomegaly', '<\\s>'), 2437),
 (('<s>', 'the'), 800),
 (('cardiothoracic', 'ratio'), 674),
 (('the', 'heart'), 587),
 (('<s>', 'there'), 571),
 (('there', 'is'), 553),
 (('ratio', 'is'), 540),
 (('is', 'no'), 532),
 (('heart', 'is'), 531)]

In [154]:
cardio_trigram_0 = createPatternNgrams(cardio_report_0, 3, cardio_keyword)
list(cardio_trigram_0.items())[:20]

[(('<s>', 'no', 'cardiomegaly'), 29563),
 (('no', 'cardiomegaly', 'is'), 27398),
 (('is', 'observed', '<\\s>'), 10757),
 (('cardiomegaly', 'is', 'observed'), 10741),
 (('is', 'seen', '<\\s>'), 9930),
 (('cardiomegaly', 'is', 'seen'), 9808),
 (('is', 'noted', '<\\s>'), 3379),
 (('cardiomegaly', 'is', 'noted'), 3289),
 (('is', 'found', '<\\s>'), 3089),
 (('cardiomegaly', 'is', 'found'), 3086),
 (('no', 'cardiomegaly', '<\\s>'), 2224),
 (('<s>', 'the', 'heart'), 571),
 (('<s>', 'there', 'is'), 548),
 (('cardiothoracic', 'ratio', 'is'), 534),
 (('there', 'is', 'no'), 516),
 (('is', 'no', 'cardiomegaly'), 497),
 (('ratio', 'is', 'about'), 465),
 (('about', '0.5', '<\\s>'), 451),
 (('is', 'about', '0.5'), 446),
 (('heart', 'is', 'not'), 403)]

## Ground truth 1

Prepare Data

In [155]:
# choose reports
cardio_df_1 = selectData(cardio_data, 'Cardiomegaly', 1.0)
cardio_df_1

,Reports,Report List,Cardiomegaly BERT Labeler,Cardiomegaly Inspectra Labeler,Cardiomegaly Ground Truth,Cardiomegaly Key Detect
0,CHEST :\nP.A. upright view .\nNo active pulmon...,"['<s> chest <\\s>', '<s> p.a <\\s>', '<s> upri...",1,0,1.0,"<re.Match object; span=(4, 24), match='lvh is ..."
5,CHEST\n\nNo definite pulmonary infiltration.\n...,"['<s> chest <\\s>', '<s> no definite pulmonary...",0,1,1.0,0.51
8,Chest;\n\n Left ventricular enlargement with ...,"['<s> chest <\\s>', '<s> left ventricular enla...",1,0,1.0,"<re.Match object; span=(4, 28), match='left ve..."
9,CHEST\n\nFibrotic change at left basal lung.\n...,"['<s> chest <\\s>', '<s> fibrotic change at le...",0,1,1.0,0.54
11,CHEST\n\nNo definite pulmonary infiltration.\n...,"['<s> chest <\\s>', '<s> no definite pulmonary...",0,1,1.0,0.54
...,...,...,...,...,...,...
39218,CHEST FILM\n\nNo pulmonary infiltration is see...,"['<s> chest film <\\s>', '<s> no pulmonary inf...",1,0,1.0,"<re.Match object; span=(4, 24), match='cardiom..."
39224,CHEST FILM\n\nNo pulmonary infiltration is see...,"['<s> chest film <\\s>', '<s> no pulmonary inf...",1,0,1.0,"<re.Match object; span=(4, 24), match='cardiom..."
39231,Chest PA upright\n \n No pulmonary infiltrat...,"['<s> chest pa upright <\\s>', '<s> no pulmona...",1,0,1.0,"<re.Match object; span=(4, 31), match='promine..."
39232,CXR PA \n\nNo definite pulmonary infiltration ...,"['<s> cxr pa <\\s>', '<s> no definite pulmonar...",0,1,1.0,0.51


In [156]:
# create report list
cardio_report_1 = createReportList(cardio_df_1)
cardio_report_1[0]

['<s> chest <\\s>',
 '<s> p.a <\\s>',
 '<s> upright view <\\s>',
 '<s> no active pulmonary disease is detected <\\s>',
 '<s> lvh is observed <\\s>',
 '<s> calcified and tortuous aortic arch are noted <\\s>']

Find N-grams Pattern

In [157]:
cardio_bigram_1 = createPatternNgrams(cardio_report_1, 2, cardio_keyword)
list(cardio_bigram_1.items())[:20]

[(('<s>', 'cardiomegaly'), 1718),
 (('cardiomegaly', 'is'), 1262),
 (('noted', '<\\s>'), 1246),
 (('is', 'noted'), 1232),
 (('cardiothoracic', 'ratio'), 975),
 (('<s>', 'prominent'), 910),
 (('ratio', 'is'), 445),
 (('ratio', '<\\s>'), 437),
 (('prominent', 'cardiac'), 328),
 (('is', 'about'), 326),
 (('prominent', 'cardiothoracic'), 313),
 (('size', '<\\s>'), 297),
 (('cardiac', 'shadow'), 296),
 (('prominent', 'heart'), 277),
 (('heart', 'size'), 250),
 (('shadow', '<\\s>'), 226),
 (('<s>', 'cardiothoracic'), 208),
 (('increased', 'cardiothoracic'), 203),
 (('cardiomegaly', '<\\s>'), 192),
 (('cardiomegaly', 'with'), 183)]

In [158]:
cardio_trigram_1 = createPatternNgrams(cardio_report_1, 3, cardio_keyword)
list(cardio_trigram_1.items())[:20]

[(('<s>', 'cardiomegaly', 'is'), 1251),
 (('is', 'noted', '<\\s>'), 1226),
 (('cardiomegaly', 'is', 'noted'), 1156),
 (('cardiothoracic', 'ratio', '<\\s>'), 436),
 (('cardiothoracic', 'ratio', 'is'), 417),
 (('<s>', 'prominent', 'cardiac'), 327),
 (('ratio', 'is', 'about'), 326),
 (('prominent', 'cardiothoracic', 'ratio'), 313),
 (('<s>', 'prominent', 'cardiothoracic'), 308),
 (('<s>', 'prominent', 'heart'), 246),
 (('prominent', 'heart', 'size'), 238),
 (('<s>', 'cardiothoracic', 'ratio'), 206),
 (('increased', 'cardiothoracic', 'ratio'), 202),
 (('cardiac', 'shadow', '<\\s>'), 198),
 (('<s>', 'cardiomegaly', 'with'), 179),
 (('prominent', 'cardiac', 'shadow'), 177),
 (('<s>', 'cardiomegaly', '<\\s>'), 173),
 (('<s>', 'increased', 'cardiothoracic'), 172),
 (('heart', 'size', '<\\s>'), 160),
 (('<s>', 'the', 'cardiothoracic'), 156)]

## Ground truth -1

Prepare Data

In [159]:
# choose reports
cardio_df_u = selectData(cardio_data, 'Cardiomegaly', -1.0)
cardio_df_u

,Reports,Report List,Cardiomegaly BERT Labeler,Cardiomegaly Inspectra Labeler,Cardiomegaly Ground Truth,Cardiomegaly Key Detect
2,CXR PA upright\nNo demonstrable active pulmona...,"['<s> cxr pa upright <\\s>', '<s> no demonstra...",0,1,-1.0,"<re.Match object; span=(4, 32), match='borderl..."
3,CXR \n\nNo clinical history is provided.\n\nNo...,"['<s> cxr <\\s>', '<s> no clinical history is ...",0,1,-1.0,"<re.Match object; span=(13, 43), match='no cha..."
6,Chest PA upright;\n Rotate chest position.\n M...,"['<s> chest pa upright <\\s>', '<s> rotate che...",0,1,-1.0,"<re.Match object; span=(37, 57), match='mild p..."
17,CXR (PA upright)\n\nNo definite pulmonary inf...,"['<s> cxr pa upright <\\s>', '<s> no definite ...",1,0,-1.0,"<re.Match object; span=(4, 26), match='mild ca..."
21,Chest film\n\nReticulonodular infiltration bot...,"['<s> chest film <\\s>', '<s> reticulonodular ...",0,1,-1.0,"<re.Match object; span=(4, 32), match='borderl..."
...,...,...,...,...,...,...
39190,Chest:PA\n\nNo active pulmonary disease.\nDila...,"['<s> chest pa <\\s>', '<s> no active pulmonar...",0,1,-1.0,"<re.Match object; span=(4, 44), match='mild in..."
39200,"CXR\n\n Compare to CXR on 12/3/2018, markedly...","['<s> cxr <\\s>', '<s> compare to cxr on 12 3 ...",0,1,-1.0,"<re.Match object; span=(4, 64), match='borderl..."
39208,CXR PA upright\n\nHistory : --\n\nComparison s...,"['<s> cxr pa upright <\\s>', '<s> history <\\s...",1,0,-1.0,"<re.Match object; span=(4, 76), match='no chan..."
39209,"CXR PA upright\n\nHistory : DM,HT \n\nCompari...","['<s> cxr pa upright <\\s>', '<s> history dm h...",1,0,-1.0,"<re.Match object; span=(4, 40), match='no chan..."


In [160]:
# create report list
cardio_report_u = createReportList(cardio_df_u)
cardio_report_u[0]

['<s> cxr pa upright <\\s>',
 '<s> no demonstrable active pulmonary infiltration <\\s>',
 '<s> borderline cardiac size <\\s>',
 '<s> bony thorax is intact <\\s>',
 '<s> both costophrenic sulci are not remarkable <\\s>',
 '<s> imp no active pulmonary disease <\\s>']

Find N-grams Pattern

In [161]:
cardio_bigram_u = createPatternNgrams(cardio_report_u, 2, cardio_keyword)
list(cardio_bigram_u.items())[:20]

[(('<s>', 'mild'), 1516),
 (('mild', 'cardiomegaly'), 881),
 (('<s>', 'borderline'), 646),
 (('mild', 'prominent'), 514),
 (('cardiomegaly', '<\\s>'), 474),
 (('<s>', 'no'), 443),
 (('prominent', 'heart'), 441),
 (('size', '<\\s>'), 372),
 (('heart', 'size'), 336),
 (('aorta', '<\\s>'), 329),
 (('cardiothoracic', 'ratio'), 325),
 (('cardiac', 'size'), 312),
 (('changed', 'of'), 311),
 (('no', 'changed'), 305),
 (('borderline', 'cardiac'), 276),
 (('cardiomegaly', 'is'), 276),
 (('pulmonary', 'vasculature'), 249),
 (('vasculature', '<\\s>'), 246),
 (('of', 'cardiomegaly'), 236),
 (('change', 'of'), 233)]

In [162]:
cardio_trigram_u = createPatternNgrams(cardio_report_u, 3, cardio_keyword)
list(cardio_trigram_u.items())[:20]

[(('<s>', 'mild', 'cardiomegaly'), 788),
 (('mild', 'prominent', 'heart'), 406),
 (('<s>', 'mild', 'prominent'), 404),
 (('prominent', 'heart', 'size'), 312),
 (('no', 'changed', 'of'), 301),
 (('<s>', 'no', 'changed'), 293),
 (('<s>', 'borderline', 'cardiac'), 276),
 (('borderline', 'cardiac', 'size'), 274),
 (('mild', 'cardiomegaly', '<\\s>'), 273),
 (('pulmonary', 'vasculature', '<\\s>'), 242),
 (('mild', 'cardiomegaly', 'is'), 233),
 (('normal', 'pulmonary', 'vasculature'), 230),
 (('heart', 'size', '<\\s>'), 218),
 (('increased', 'cardiothoracic', 'ratio'), 202),
 (('no', 'change', 'of'), 182),
 (('<s>', 'borderline', 'cardiomegaly'), 173),
 (('mild', 'cardiomegaly', 'normal'), 168),
 (('cardiomegaly', 'normal', 'pulmonary'), 164),
 (('cardiothoracic', 'ratio', '<\\s>'), 163),
 (('cardiac', 'size', 'with'), 157)]

## Unlabel

Prepare Data

In [163]:
# choose reports
cardio_df = selectData(cardio_data, 'Cardiomegaly', None)
cardio_df

,Reports,Report List,Cardiomegaly BERT Labeler,Cardiomegaly Inspectra Labeler,Cardiomegaly Ground Truth,Cardiomegaly Key Detect
4,Chest PA upright;\n R/O lung metastasis.\n\n E...,"['<s> chest pa upright <\\s>', '<s> r o lung m...",1,0,NaN,NaN
7,Chest PA uprihgt;\n Rt.lung destruction with f...,"['<s> chest pa upright <\\s>', '<s> rt. lung d...",0,1,NaN,NaN
12,CXR PA upright\n The chest shows no definite ...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,NaN,NaN
15,CHEST PA UPRIGHT\n\nHistory: not available\nco...,"['<s> chest pa upright <\\s>', '<s> history no...",0,1,NaN,NaN
22,CHEST PA UPRIGHT\n\nHistory: not available\n\n...,"['<s> chest pa upright <\\s>', '<s> history no...",1,0,NaN,NaN
...,...,...,...,...,...,...
38828,CXR PA\n\nThere is right side aortic arch with...,"['<s> cxr pa <\\s>', '<s> there is right side ...",1,0,NaN,NaN
38882,CXR PA\n\nGlobular heart shape is noted.\nBlun...,"['<s> cxr pa <\\s>', '<s> globular heart shape...",0,1,NaN,NaN
39016,Chest:PA\nPost thoracic surgery with prostheti...,"['<s> chest pa <\\s>', '<s> post thoracic surg...",1,0,NaN,NaN
39175,CXR PA UPRIGHT\n\nInterstitial infiltration in...,"['<s> cxr pa upright <\\s>', '<s> interstitial...",0,1,NaN,NaN


In [164]:
# create report list
cardio_report = createReportList(cardio_df)
cardio_report[0]

['<s> chest pa upright <\\s>',
 '<s> r o lung metastasis <\\s>',
 '<s> enlarged size and increased number of diffuse pulmonary nodules in both lungs are seen <\\s>',
 '<s> much more rt.pleural effusion is also shown <\\s>',
 '<s> ds <\\s>']

Find N-grams Pattern

In [165]:
cardio_bigram = createPatternNgrams(cardio_report, 2)
list(cardio_bigram.items())[:20]

[(('<s>', 'no'), 481),
 (('pleural', 'effusion'), 270),
 (('<s>', 'the'), 248),
 (('seen', '<\\s>'), 227),
 (('noted', '<\\s>'), 218),
 (('pulmonary', 'infiltration'), 214),
 (('intact', '<\\s>'), 213),
 (('bony', 'thorax'), 211),
 (('is', 'seen'), 199),
 (('is', 'noted'), 193),
 (('clear', '<\\s>'), 161),
 (('is', 'intact'), 158),
 (('thorax', 'is'), 157),
 (('effusion', '<\\s>'), 151),
 (('<s>', 'bony'), 147),
 (('<s>', 'there'), 142),
 (('active', 'pulmonary'), 135),
 (('there', 'is'), 134),
 (('effusion', 'is'), 133),
 (('no', 'active'), 128)]

In [166]:
cardio_trigram = createPatternNgrams(cardio_report, 3)
list(cardio_trigram.items())[:20]

[(('is', 'seen', '<\\s>'), 180),
 (('is', 'noted', '<\\s>'), 164),
 (('is', 'intact', '<\\s>'), 157),
 (('bony', 'thorax', 'is'), 154),
 (('thorax', 'is', 'intact'), 140),
 (('pleural', 'effusion', '<\\s>'), 128),
 (('<s>', 'there', 'is'), 121),
 (('are', 'clear', '<\\s>'), 112),
 (('no', 'pleural', 'effusion'), 110),
 (('<s>', 'bony', 'thorax'), 110),
 (('active', 'pulmonary', 'infiltration'), 108),
 (('<s>', 'no', 'pleural'), 103),
 (('no', 'active', 'pulmonary'), 102),
 (('pulmonary', 'infiltration', '<\\s>'), 100),
 (('<s>', 'no', 'active'), 100),
 (('pleural', 'effusion', 'is'), 95),
 (('<s>', 'both', 'costophrenic'), 85),
 (('costophrenic', 'angles', 'are'), 81),
 (('angles', 'are', 'clear'), 72),
 (('effusion', 'is', 'seen'), 70)]

In [167]:
cardio_fourgram = createPatternNgrams(cardio_report, 4)
list(cardio_fourgram.items())[:20]

[(('thorax', 'is', 'intact', '<\\s>'), 139),
 (('bony', 'thorax', 'is', 'intact'), 137),
 (('<s>', 'no', 'pleural', 'effusion'), 102),
 (('<s>', 'bony', 'thorax', 'is'), 94),
 (('<s>', 'no', 'active', 'pulmonary'), 82),
 (('no', 'active', 'pulmonary', 'infiltration'), 79),
 (('costophrenic', 'angles', 'are', 'clear'), 72),
 (('angles', 'are', 'clear', '<\\s>'), 72),
 (('effusion', 'is', 'seen', '<\\s>'), 70),
 (('no', 'definite', 'pulmonary', 'infiltration'), 58),
 (('both', 'costophrenic', 'angles', 'are'), 57),
 (('no', 'pleural', 'effusion', 'is'), 57),
 (('active', 'pulmonary', 'infiltration', '<\\s>'), 53),
 (('<s>', 'both', 'costophrenic', 'angles'), 50),
 (('pleural', 'effusion', 'is', 'seen'), 49),
 (('no', 'pleural', 'effusion', '<\\s>'), 49),
 (('<s>', 'the', 'bony', 'thorax'), 48),
 (('the', 'bony', 'thorax', 'is'), 48),
 (('<s>', 'there', 'is', 'no'), 37),
 (('pulmonary', 'infiltration', 'or', 'nodule'), 37)]

In [168]:
cardio_bigramk = createPatternNgrams(cardio_report, 2, cardio_keyword)
list(cardio_bigramk.items())[:20]

[(('<s>', 'heart'), 111),
 (('heart', 'and'), 65),
 (('<s>', 'the'), 64),
 (('the', 'heart'), 51),
 (('and', 'mediastinum'), 40),
 (('heart', 'size'), 39),
 (('to', 'the'), 37),
 (('heart', 'is'), 33),
 (('<s>', 'there'), 31),
 (('there', 'is'), 31),
 (('is', 'noted'), 27),
 (('of', 'heart'), 25),
 (('the', 'left'), 25),
 (('cannot', 'be'), 24),
 (('cardiac', 'shadow'), 24),
 (('<s>', 'unchanged'), 24),
 (('noted', '<\\s>'), 24),
 (('size', '<\\s>'), 23),
 (('and', 'great'), 22),
 (('great', 'vessels'), 22)]

In [169]:
cardio_trigramk = createPatternNgrams(cardio_report, 3, cardio_keyword)
list(cardio_trigramk.items())[:20]

[(('<s>', 'the', 'heart'), 48),
 (('heart', 'and', 'mediastinum'), 36),
 (('<s>', 'heart', 'and'), 36),
 (('<s>', 'there', 'is'), 29),
 (('<s>', 'heart', 'size'), 26),
 (('the', 'heart', 'and'), 22),
 (('heart', 'and', 'great'), 22),
 (('and', 'great', 'vessels'), 21),
 (('heart', 'size', 'cannot'), 20),
 (('size', 'cannot', 'be'), 19),
 (('is', 'noted', '<\\s>'), 19),
 (('to', 'the', 'left'), 19),
 (('and', 'mediastinum', 'are'), 18),
 (('there', 'is', 'cardiomegaly'), 18),
 (('to', 'the', 'right'), 17),
 (('shift', 'to', 'the'), 16),
 (('<s>', 'heart', 'is'), 14),
 (('are', 'unremarkable', '<\\s>'), 14),
 (('great', 'vessels', 'are'), 13),
 (('is', 'seen', '<\\s>'), 13)]

In [170]:
cardio_fourgramk = createPatternNgrams(cardio_report, 4, cardio_keyword)
list(cardio_fourgramk.items())[:20]

[(('<s>', 'the', 'heart', 'and'), 21),
 (('heart', 'and', 'great', 'vessels'), 21),
 (('<s>', 'heart', 'and', 'mediastinum'), 21),
 (('heart', 'size', 'cannot', 'be'), 19),
 (('<s>', 'there', 'is', 'cardiomegaly'), 18),
 (('<s>', 'heart', 'size', 'cannot'), 16),
 (('heart', 'and', 'mediastinum', 'are'), 15),
 (('and', 'great', 'vessels', 'are'), 13),
 (('the', 'heart', 'and', 'great'), 11),
 (('and', 'mediastinum', 'are', 'unremarkable'), 11),
 (('mediastinum', 'are', 'unremarkable', '<\\s>'), 11),
 (('<s>', 'heart', 'and', 'great'), 11),
 (('the', 'heart', 'and', 'mediastinum'), 10),
 (('heart', 'and', 'mediastinum', 'shift'), 10),
 (('there', 'is', 'cardiomegaly', 'and'), 10),
 (('is', 'cardiomegaly', 'and', 'tortuous'), 10),
 (('cannot', 'be', 'well', 'evaluated'), 10),
 (('great', 'vessels', 'are', 'intact'), 9),
 (('vessels', 'are', 'intact', '<\\s>'), 9),
 (('and', 'mediastinum', 'shift', 'to'), 9)]

In [171]:
cardio_fivegramk = createPatternNgrams(cardio_report, 5, cardio_keyword)
list(cardio_fivegramk.items())[:20]

[(('<s>', 'heart', 'size', 'cannot', 'be'), 15),
 (('heart', 'and', 'great', 'vessels', 'are'), 13),
 (('<s>', 'the', 'heart', 'and', 'great'), 11),
 (('heart', 'and', 'mediastinum', 'are', 'unremarkable'), 11),
 (('and', 'mediastinum', 'are', 'unremarkable', '<\\s>'), 11),
 (('<s>', 'heart', 'and', 'great', 'vessels'), 11),
 (('the', 'heart', 'and', 'great', 'vessels'), 10),
 (('<s>', 'there', 'is', 'cardiomegaly', 'and'), 10),
 (('there', 'is', 'cardiomegaly', 'and', 'tortuous'), 10),
 (('and', 'great', 'vessels', 'are', 'intact'), 9),
 (('great', 'vessels', 'are', 'intact', '<\\s>'), 9),
 (('<s>', 'the', 'heart', 'and', 'mediastinum'), 9),
 (('<s>', 'heart', 'and', 'mediastinum', 'shift'), 9),
 (('heart', 'and', 'mediastinum', 'shift', 'to'), 9),
 (('and', 'mediastinum', 'shift', 'to', 'the'), 9),
 (('to', 'the', 'left', 'side', '<\\s>'), 9),
 (('<s>', 'heart', 'and', 'mediastinum', 'are'), 9),
 (('is', 'cardiomegaly', 'and', 'tortuous', 'aorta'), 9),
 (('cardiomegaly', 'and', 'tort

In [172]:
cardio_sixgramk = createPatternNgrams(cardio_report, 6, cardio_keyword)
list(cardio_sixgramk.items())[:20]

[(('heart', 'and', 'mediastinum', 'are', 'unremarkable', '<\\s>'), 11),
 (('<s>', 'the', 'heart', 'and', 'great', 'vessels'), 10),
 (('<s>', 'there', 'is', 'cardiomegaly', 'and', 'tortuous'), 10),
 (('heart', 'and', 'great', 'vessels', 'are', 'intact'), 9),
 (('and', 'great', 'vessels', 'are', 'intact', '<\\s>'), 9),
 (('<s>', 'heart', 'and', 'mediastinum', 'shift', 'to'), 9),
 (('heart', 'and', 'mediastinum', 'shift', 'to', 'the'), 9),
 (('<s>', 'heart', 'and', 'mediastinum', 'are', 'unremarkable'), 9),
 (('there', 'is', 'cardiomegaly', 'and', 'tortuous', 'aorta'), 9),
 (('is', 'cardiomegaly', 'and', 'tortuous', 'aorta', '<\\s>'), 9),
 (('<s>', 'heart', 'size', 'cannot', 'be', 'well'), 8),
 (('heart', 'size', 'cannot', 'be', 'well', 'evaluated'), 8),
 (('the', 'heart', 'and', 'great', 'vessels', 'are'), 7),
 (('enabled', 'cardiac', 'shadow', 'is', 'shown', '<\\s>'), 7),
 (('shift', 'to', 'the', 'left', 'side', '<\\s>'), 7),
 (('heart', 'size', 'cannot', 'be', 'evaluated', '<\\s>'), 6)

## Difference between Result 2 and 3

In [173]:
findDiffData('Cardiomegaly', cardio_data_old, cardio_data)

nan 1.0 ['<s> cxr pa upright <\\s>', '<s> lung clear <\\s>', '<s> cardiomegaly with mild tours of the aorta are noted <\\s>', '<s> no pleural effusion <\\s>', '<s> bony structures are intact <\\s>']
0.0 nan ['<s> chest pa upright <\\s>', '<s> interstitial pulmonary marking thickening at both upper lungs are shown <\\s>', '<s> the rest of bothlungs are clear <\\s>', '<s> no pleural effusion is seen <\\s>', '<s> heart is formalin size <\\s>', '<s> tortuous thoracic aorta is evident <\\s>', '<s> no pleural effusion is apparent <\\s>', '<s> imp interstitial pulmonary marking thickening at both upper lungs infiltration is probable <\\s>', '<s> ds <\\s>']
nan -1.0 ['<s> cxr pa upright <\\s>', '<s> minimal increase interstitial density right middle lung <\\s>', '<s> borderline cardiac size <\\s>', '<s> bony thorax is intact <\\s>', '<s> both costophrenic sulci are not remarkable <\\s>']
nan -1.0 ['<s> cxr <\\s>', '<s> borderline cardiomegaly with pulmonary congestion <\\s>', '<s> atherosclero

In [174]:
findDiffData2('Cardiomegaly', cardio_data_old, cardio_data)

nan 1.0 ['<s> cxr pa upright <\\s>', '<s> lung clear <\\s>', '<s> cadiomegaly with mild totuos of the aorta are noted <\\s>', '<s> no pleural effusion <\\s>', '<s> bony structures are intact <\\s>']
         ['<s> cxr pa upright <\\s>', '<s> lung clear <\\s>', '<s> cardiomegaly with mild tours of the aorta are noted <\\s>', '<s> no pleural effusion <\\s>', '<s> bony structures are intact <\\s>']
0.0 nan ['<s> chest pa uprihgt <\\s>', '<s> interstiital pulmonary marking thickening at both upper lungs are shown <\\s>', '<s> the rest of bothlungs are clear <\\s>', '<s> no pleural effusion is seen <\\s>', '<s> heart is normalin size <\\s>', '<s> tortuous throacic aorta is evident <\\s>', '<s> no pleurla effusion is apparent <\\s>', '<s> imp intesritial pulmonary marking thickening at both upper lungs infiltration is probable <\\s>', '<s> ds <\\s>']
         ['<s> chest pa upright <\\s>', '<s> interstitial pulmonary marking thickening at both upper lungs are shown <\\s>', '<s> the rest of b

# Pleural Effusion

In [175]:
# define keyword
pleural_keyword = r'pleural|costophrenic'

## Ground truth 0

Prepare Data

In [176]:
# choose reports
pleural_df_0 = selectData(pleural_data, 'Pleural Effusion', 0.0)
pleural_df_0

,Reports,Report List,Pleural Effusion BERT Labeler,Pleural Effusion Inspectra Labeler,Pleural Effusion Ground Truth,Pleural Effusion Key Detect
7,CXR\nInterstitial infitration both lungs\nMild...,"['<s> cxr <\\s>', '<s> interstitial infiltrati...",1,0,0.0,"<re.Match object; span=(0, 36), match='<s> bot..."
8,Chest PA upright;\n compared to film on 20/4/2...,"['<s> chest pa upright <\\s>', '<s> compared t...",1,0,0.0,"<re.Match object; span=(0, 33), match='<s> rt...."
11,CXR PA upright (13/7/2005)\n\n Fibros...,"['<s> cxr pa upright 13 7 2005 <\\s>', '<s> fi...",1,0,0.0,"<re.Match object; span=(0, 36), match='<s> bot..."
18,Chest film\n\nMinimal fibrosis in RUL and inte...,"['<s> chest film <\\s>', '<s> minimal fibrosis...",1,0,0.0,"<re.Match object; span=(0, 36), match='<s> bot..."
21,CXR PA upright\n By compared with previous ch...,"['<s> cxr pa upright <\\s>', '<s> by compared ...",1,0,0.0,"<re.Match object; span=(0, 43), match='<s> bot..."
...,...,...,...,...,...,...
4544,CXR PA \n\nMinimal fibrosis at LLL.\nNormal ca...,"['<s> cxr pa <\\s>', '<s> minimal fibrosis at ...",1,0,0.0,"<re.Match object; span=(0, 36), match='<s> bot..."
4547,CXR\n\nAs compared to prior film on 12 march 1...,"['<s> cxr <\\s>', '<s> as compared to prior fi...",0,1,0.0,"<re.Match object; span=(58, 102), match='no de..."
4551,CXR PA upright\n\n Compared to prior film on 3...,"['<s> cxr pa upright <\\s>', '<s> compared to ...",1,0,0.0,"<re.Match object; span=(4, 45), match='no defi..."
4558,CXR PA upright\n\nComparison to prior radiogra...,"['<s> cxr pa upright <\\s>', '<s> comparison t...",0,1,0.0,"<re.Match object; span=(4, 147), match='no int..."


In [177]:
# create report list
pleural_report_0 = createReportList(pleural_df_0)
pleural_report_0[0]

['<s> cxr <\\s>',
 '<s> interstitial infiltration both lungs <\\s>',
 '<s> mild cardiomegaly <\\s>',
 '<s> both costophrenic sulci are not remarkable <\\s>']

Find N-grams Pattern

In [178]:
pleural_bigram_0 = createPatternNgrams(pleural_report_0, 2, pleural_keyword)
list(pleural_bigram_0.items())[:20]

[(('pleural', 'effusion'), 666),
 (('<s>', 'both'), 430),
 (('both', 'costophrenic'), 420),
 (('clear', '<\\s>'), 366),
 (('are', 'clear'), 331),
 (('effusion', '<\\s>'), 268),
 (('<s>', 'neither'), 264),
 (('neither', 'pleural'), 263),
 (('effusion', 'nor'), 263),
 (('nor', 'pneumothorax'), 263),
 (('pneumothorax', '<\\s>'), 263),
 (('costophrenic', 'angles'), 244),
 (('angles', 'are'), 244),
 (('<s>', 'no'), 244),
 (('change', 'of'), 228),
 (('costophrenic', 'sulci'), 220),
 (('sulci', 'are'), 179),
 (('right', 'pleural'), 168),
 (('are', 'not'), 164),
 (('not', 'remarkable'), 164)]

In [179]:
pleural_trigram_0 = createPatternNgrams(pleural_report_0, 3, pleural_keyword)
list(pleural_trigram_0.items())[:20]

[(('<s>', 'both', 'costophrenic'), 416),
 (('are', 'clear', '<\\s>'), 331),
 (('<s>', 'neither', 'pleural'), 263),
 (('neither', 'pleural', 'effusion'), 263),
 (('pleural', 'effusion', 'nor'), 263),
 (('effusion', 'nor', 'pneumothorax'), 263),
 (('nor', 'pneumothorax', '<\\s>'), 262),
 (('pleural', 'effusion', '<\\s>'), 258),
 (('costophrenic', 'angles', 'are'), 244),
 (('angles', 'are', 'clear'), 241),
 (('both', 'costophrenic', 'angles'), 236),
 (('costophrenic', 'sulci', 'are'), 179),
 (('both', 'costophrenic', 'sulci'), 178),
 (('are', 'not', 'remarkable'), 164),
 (('not', 'remarkable', '<\\s>'), 164),
 (('right', 'pleural', 'effusion'), 160),
 (('no', 'interval', 'change'), 130),
 (('interval', 'change', 'of'), 130),
 (('sulci', 'are', 'not'), 129),
 (('<s>', 'no', 'interval'), 120)]

## Ground truth 1

Prepare Data

In [180]:
# choose reports
pleural_df_1 = selectData(pleural_data, 'Pleural Effusion', 1.0)
pleural_df_1

,Reports,Report List,Pleural Effusion BERT Labeler,Pleural Effusion Inspectra Labeler,Pleural Effusion Ground Truth,Pleural Effusion Key Detect
4,CXR PA upright\n\nPulmonary infiltration at RL...,"['<s> cxr pa upright <\\s>', '<s> pulmonary in...",0,1,1.0,"<re.Match object; span=(38, 60), match='right ..."
9,CHEST PA UPRIGHT\n\nRight pleural effusion and...,"['<s> chest pa upright <\\s>', '<s> right pleu...",1,0,1.0,"<re.Match object; span=(4, 26), match='right p..."
10,CHEST PA UPRIGHT\n\nThere is righ tpleural eff...,"['<s> chest pa upright <\\s>', '<s> there is h...",1,0,1.0,"<re.Match object; span=(0, 34), match='<s> the..."
17,CXR (PA_upright)\n\nFibrosis with reticulonod...,"['<s> cxr pa_upright <\\s>', '<s> fibrosis wit...",0,1,1.0,"<re.Match object; span=(4, 26), match='right p..."
25,CXR(PA)\n \nClinical history: loculated ple...,"['<s> cxr pa <\\s>', '<s> clinical history loc...",1,0,1.0,"<re.Match object; span=(0, 45), match='<s> cli..."
...,...,...,...,...,...,...
4497,Chest PA upright \n\nHistory: Rt.pleural effus...,"['<s> chest pa upright <\\s>', '<s> history rt...",1,0,1.0,"<re.Match object; span=(0, 31), match='<s> his..."
4549,CXR PA upright\n\nLarge bleb at RUL\nOpacity a...,"['<s> cxr pa upright <\\s>', '<s> large bleb a...",0,1,1.0,"<re.Match object; span=(4, 26), match='right p..."
4555,Chest PA upright \n\nPrior noted bilateral ple...,"['<s> chest pa upright <\\s>', '<s> prior note...",1,0,1.0,"<re.Match object; span=(10, 42), match='noted ..."
4556,CHEST FILM\n\nReticulonodular infiltration at ...,"['<s> chest film <\\s>', '<s> reticulonodular ...",1,0,1.0,"<re.Match object; span=(20, 41), match='left p..."


In [181]:
# create report list
pleural_report_1 = createReportList(pleural_df_1)
pleural_report_1[0]

['<s> cxr pa upright <\\s>',
 '<s> pulmonary infiltration at rll and right pleural effusion <\\s>',
 '<s> normal cardiothoracic ratio <\\s>',
 '<s> bony thorax is intact <\\s>']

Find N-grams Pattern

In [182]:
pleural_bigram_1 = createPatternNgrams(pleural_report_1, 2, pleural_keyword)
list(pleural_bigram_1.items())[:20]

[(('pleural', 'effusion'), 401),
 (('effusion', '<\\s>'), 307),
 (('right', 'pleural'), 169),
 (('left', 'pleural'), 117),
 (('<s>', 'right'), 81),
 (('bilateral', 'pleural'), 67),
 (('<s>', 'left'), 63),
 (('effusion', 'is'), 58),
 (('<s>', 'bilateral'), 42),
 (('<s>', 'small'), 33),
 (('and', 'right'), 32),
 (('is', 'seen'), 30),
 (('costophrenic', 'angle'), 30),
 (('infiltration', 'at'), 29),
 (('seen', '<\\s>'), 29),
 (('noted', '<\\s>'), 29),
 (('<s>', 'no'), 29),
 (('<s>', 'pulmonary'), 25),
 (('pulmonary', 'infiltration'), 25),
 (('is', 'noted'), 25)]

In [183]:
pleural_trigram_1 = createPatternNgrams(pleural_report_1, 3, pleural_keyword)
list(pleural_trigram_1.items())[:20]

[(('pleural', 'effusion', '<\\s>'), 304),
 (('right', 'pleural', 'effusion'), 165),
 (('left', 'pleural', 'effusion'), 117),
 (('<s>', 'right', 'pleural'), 74),
 (('pleural', 'effusion', 'is'), 57),
 (('bilateral', 'pleural', 'effusion'), 56),
 (('<s>', 'left', 'pleural'), 54),
 (('<s>', 'bilateral', 'pleural'), 39),
 (('and', 'right', 'pleural'), 28),
 (('is', 'seen', '<\\s>'), 28),
 (('effusion', 'is', 'seen'), 26),
 (('<s>', 'pulmonary', 'infiltration'), 25),
 (('pulmonary', 'infiltration', 'at'), 24),
 (('is', 'noted', '<\\s>'), 23),
 (('costophrenic', 'angle', 'is'), 23),
 (('infiltration', 'at', 'rll'), 21),
 (('of', 'right', 'pleural'), 20),
 (('at', 'rll', 'and'), 19),
 (('rll', 'and', 'right'), 19),
 (('left', 'costophrenic', 'angle'), 17)]

## Ground truth -1

Prepare Data

In [184]:
# choose reports
pleural_df_u = selectData(pleural_data, 'Pleural Effusion', -1.0)
pleural_df_u

,Reports,Report List,Pleural Effusion BERT Labeler,Pleural Effusion Inspectra Labeler,Pleural Effusion Ground Truth,Pleural Effusion Key Detect
0,"Chest PA upright;\n follow up film , compared ...","['<s> chest pa upright <\\s>', '<s> follow up ...",0,1,-1.0,"<re.Match object; span=(4, 49), match='no sign..."
1,Chest;\n\n Minimal fibrosis at RUL .\n The l...,"['<s> chest <\\s>', '<s> minimal fibrosis at r...",1,0,-1.0,"<re.Match object; span=(0, 42), match='<s> bot..."
2,CHEST\n\nReticulonodular infiltration at RUL. ...,"['<s> chest <\\s>', '<s> reticulonodular infil...",0,1,-1.0,"<re.Match object; span=(4, 48), match='no sign..."
5,CXR\n\nHistory of CA lung\n\nAs compared to pr...,"['<s> cxr <\\s>', '<s> history of ca lung <\\s...",0,1,-1.0,"<re.Match object; span=(56, 91), match='no cha..."
12,CXR(PA)\n \n History: Pleural TB. Foll...,"['<s> cxr pa <\\s>', '<s> history pleural tb. ...",0,1,-1.0,"<re.Match object; span=(49, 100), match='no si..."
...,...,...,...,...,...,...
4548,"CXR\n\n Compare to CXR on 30/04/2017, no si...","['<s> cxr <\\s>', '<s> compare to cxr on 30 04...",0,1,-1.0,"<re.Match object; span=(33, 80), match='no sig..."
4550,CXR PA\n\nCompared with previous study: Jan 2...,"['<s> cxr pa <\\s>', '<s> compared with previo...",0,1,-1.0,"<re.Match object; span=(4, 53), match='no sign..."
4552,CXR PA upright\n\n Compared with previous che...,"['<s> cxr pa upright <\\s>', '<s> compared wit...",0,1,-1.0,"<re.Match object; span=(20, 63), match='no cha..."
4553,CXR PA upright\n\n Compared with previous che...,"['<s> cxr pa upright <\\s>', '<s> compared wit...",0,1,-1.0,"<re.Match object; span=(20, 59), match='no cha..."


In [185]:
# create report list
pleural_report_u = createReportList(pleural_df_u)
pleural_report_u[0]

['<s> chest pa upright <\\s>',
 '<s> follow up film compared to film on 22 3 2005 <\\s>',
 '<s> no significant changed of rt.pleural effusion opacity of rll <\\s>',
 '<s> l.t lung is clear <\\s>',
 '<s> remaining thickening of rt.paratracheal soft tissue is also shown <\\s>',
 '<s> heart is mild prominent in size <\\s>',
 '<s> tortuous thoracic aorta is seen <\\s>',
 '<s> ds <\\s>']

Find N-grams Pattern

In [186]:
pleural_bigram_u = createPatternNgrams(pleural_report_u, 2, pleural_keyword)
list(pleural_bigram_u.items())[:20]

[(('pleural', 'effusion'), 2393),
 (('change', 'of'), 1918),
 (('<s>', 'no'), 1729),
 (('effusion', '<\\s>'), 1443),
 (('no', 'change'), 1133),
 (('right', 'pleural'), 1130),
 (('no', 'significant'), 856),
 (('significant', 'change'), 789),
 (('left', 'pleural'), 782),
 (('of', 'right'), 635),
 (('costophrenic', 'angle'), 594),
 (('of', 'left'), 447),
 (('pleural', 'thickening'), 394),
 (('left', 'costophrenic'), 363),
 (('<s>', 'the'), 352),
 (('there', 'is'), 349),
 (('effusion', 'is'), 342),
 (('clear', '<\\s>'), 331),
 (('is', 'no'), 310),
 (('angle', 'is'), 297)]

In [187]:
pleural_trigram_u = createPatternNgrams(pleural_report_u, 3, pleural_keyword)
list(pleural_trigram_u.items())[:20]

[(('pleural', 'effusion', '<\\s>'), 1385),
 (('right', 'pleural', 'effusion'), 1100),
 (('no', 'change', 'of'), 1096),
 (('no', 'significant', 'change'), 789),
 (('significant', 'change', 'of'), 779),
 (('left', 'pleural', 'effusion'), 762),
 (('<s>', 'no', 'change'), 681),
 (('<s>', 'no', 'significant'), 636),
 (('of', 'right', 'pleural'), 516),
 (('change', 'of', 'right'), 436),
 (('of', 'left', 'pleural'), 334),
 (('pleural', 'effusion', 'is'), 331),
 (('left', 'costophrenic', 'angle'), 330),
 (('there', 'is', 'no'), 306),
 (('change', 'of', 'left'), 287),
 (('costophrenic', 'angle', 'is'), 287),
 (('<s>', 'there', 'is'), 273),
 (('right', 'costophrenic', 'angle'), 249),
 (('pleural', 'effusion', 'and'), 245),
 (('bilateral', 'pleural', 'effusion'), 239)]

## Unlabel

Prepare Data

In [188]:
# choose reports
pleural_df = selectData(pleural_data, 'Pleural Effusion', None)
pleural_df

,Reports,Report List,Pleural Effusion BERT Labeler,Pleural Effusion Inspectra Labeler,Pleural Effusion Ground Truth,Pleural Effusion Key Detect
3,CHEST\n\nRt hydropneumothorax is seen.\nLeft l...,"['<s> chest <\\s>', '<s> rt hydropneumothorax ...",0,1,NaN,NaN
6,CHEST\n\nThere is both upper lobes infiltratio...,"['<s> chest <\\s>', '<s> there is both upper l...",1,0,NaN,NaN
13,CHEST.;\nLoss of right breast shadow.\nMassive...,"['<s> chest <\\s>', '<s> loss of right breast ...",1,0,NaN,NaN
15,CHEST\n\nS/P sternotomy.\nCompared to previous...,"['<s> chest <\\s>', '<s> s p sternotomy <\\s>'...",0,1,NaN,NaN
19,Chest ( PA upright view)\n As compared with ...,"['<s> chest pa upright view <\\s>', '<s> as co...",0,1,NaN,NaN
...,...,...,...,...,...,...
4494,Chest PA upright\n\nFindings:\nNo cahnge of le...,"['<s> chest pa upright <\\s>', '<s> findings <...",0,1,NaN,NaN
4502,CXR PA upright\n\nHistory : --\n\nComparison s...,"['<s> cxr pa upright <\\s>', '<s> history <\\s...",0,1,NaN,NaN
4545,Chest PA upright\n\nComparison: 3/4/2018\n\nFi...,"['<s> chest pa upright <\\s>', '<s> comparison...",0,1,NaN,NaN
4554,Chest PA upright \n\ncompared to film on 25/1/...,"['<s> chest pa upright <\\s>', '<s> compared t...",0,1,NaN,NaN


In [189]:
# create report list
pleural_report = createReportList(pleural_df)
pleural_report[0]

['<s> chest <\\s>',
 '<s> rt hydropneumothorax is seen <\\s>',
 '<s> left lung is clear <\\s>',
 '<s> no cardiomegaly is noted <\\s>']

In [190]:
# define keyword
pleural_keyword = r'pleural|costophrenic|effus|costoph|pleu'

Find N-grams Pattern

In [191]:
pleural_bigram = createPatternNgrams(pleural_report, 2)
list(pleural_bigram.items())[:20]

[(('<s>', 'no'), 295),
 (('change', 'of'), 202),
 (('<s>', 'there'), 199),
 (('<s>', 'the'), 195),
 (('there', 'is'), 177),
 (('no', 'change'), 154),
 (('noted', '<\\s>'), 136),
 (('seen', '<\\s>'), 130),
 (('intact', '<\\s>'), 122),
 (('lung', '<\\s>'), 119),
 (('left', 'lung'), 118),
 (('bony', 'thorax'), 117),
 (('is', 'noted'), 112),
 (('of', 'the'), 111),
 (('is', 'seen'), 109),
 (('lung', 'is'), 101),
 (('at', 'right'), 99),
 (('<s>', 'bony'), 96),
 (('at', 'the'), 94),
 (('at', 'left'), 93)]

In [192]:
pleural_trigram = createPatternNgrams(pleural_report, 3)
list(pleural_trigram.items())[:20]

[(('<s>', 'there', 'is'), 169),
 (('no', 'change', 'of'), 116),
 (('is', 'noted', '<\\s>'), 95),
 (('is', 'seen', '<\\s>'), 94),
 (('is', 'intact', '<\\s>'), 81),
 (('bony', 'thorax', 'is'), 78),
 (('thorax', 'is', 'intact'), 70),
 (('<s>', 'bony', 'thorax'), 68),
 (('lung', 'is', 'clear'), 67),
 (('no', 'significant', 'change'), 67),
 (('is', 'clear', '<\\s>'), 66),
 (('<s>', 'no', 'change'), 65),
 (('<s>', 'no', 'cardiomegaly'), 60),
 (('left', 'lung', 'is'), 59),
 (('significant', 'change', 'of'), 50),
 (('there', 'is', 'no'), 49),
 (('not', 'enlarged', '<\\s>'), 48),
 (('fibrosis', 'at', 'the'), 48),
 (('is', 'not', 'enlarged'), 47),
 (('<s>', 'as', 'compared'), 46)]

In [193]:
pleural_fourgram = createPatternNgrams(pleural_report, 4)
list(pleural_fourgram.items())[:20]

[(('bony', 'thorax', 'is', 'intact'), 70),
 (('thorax', 'is', 'intact', '<\\s>'), 70),
 (('lung', 'is', 'clear', '<\\s>'), 59),
 (('<s>', 'no', 'change', 'of'), 55),
 (('no', 'significant', 'change', 'of'), 50),
 (('is', 'not', 'enlarged', '<\\s>'), 46),
 (('<s>', 'bony', 'thorax', 'is'), 46),
 (('<s>', 'there', 'is', 'no'), 46),
 (('left', 'lung', 'is', 'clear'), 45),
 (('heart', 'is', 'not', 'enlarged'), 41),
 (('<s>', 'left', 'lung', 'is'), 40),
 (('<s>', 'no', 'significant', 'change'), 40),
 (('<s>', 'the', 'heart', 'is'), 36),
 (('<s>', 'no', 'cardiomegaly', '<\\s>'), 34),
 (('<s>', 'the', 'chest', 'shows'), 30),
 (('there', 'is', 'no', 'change'), 30),
 (('is', 'no', 'change', 'of'), 30),
 (('<s>', 'there', 'is', 'fibrosis'), 29),
 (('<s>', 'the', 'bony', 'thorax'), 28),
 (('the', 'bony', 'thorax', 'is'), 28)]

In [194]:
pleural_bigramk = createPatternNgrams(pleural_report, 2, pleural_keyword)
list(pleural_bigramk.items())[:20]

[(('change', 'of'), 97),
 (('no', 'change'), 77),
 (('effusion', '<\\s>'), 72),
 (('<s>', 'no'), 70),
 (('pleural', 'thickening'), 67),
 (('there', 'is'), 42),
 (('<s>', 'there'), 41),
 (('pleural', 'fusion'), 40),
 (('of', 'the'), 39),
 (('left', 'pleural'), 35),
 (('left', 'effusion'), 31),
 (('of', 'right'), 30),
 (('right', 'effusion'), 28),
 (('lung', '<\\s>'), 25),
 (('<s>', 'left'), 25),
 (('no', 'significant'), 24),
 (('right', 'pleural'), 22),
 (('significant', 'change'), 22),
 (('thickening', '<\\s>'), 22),
 (('bilateral', 'pleural'), 22)]

In [195]:
pleural_trigramk = createPatternNgrams(pleural_report, 3, pleural_keyword)
list(pleural_trigramk.items())[:20]

[(('no', 'change', 'of'), 74),
 (('<s>', 'no', 'change'), 39),
 (('<s>', 'there', 'is'), 38),
 (('no', 'significant', 'change'), 22),
 (('change', 'of', 'the'), 22),
 (('there', 'is', 'no'), 22),
 (('pleural', 'thickening', '<\\s>'), 21),
 (('pleural', 'fusion', '<\\s>'), 20),
 (('significant', 'change', 'of'), 18),
 (('is', 'no', 'change'), 17),
 (('left', 'effusion', '<\\s>'), 17),
 (('<s>', 'no', 'significant'), 16),
 (('left', 'pleural', 'fusion'), 16),
 (('<s>', 'the', 'chest'), 15),
 (('the', 'chest', 'shows'), 15),
 (('effusion', 'compared', 'to'), 13),
 (('change', 'of', 'left'), 13),
 (('chest', 'shows', 'no'), 12),
 (('pleural', 'thickening', 'at'), 12),
 (('right', 'effusion', '<\\s>'), 12)]

In [196]:
pleural_fourgramk = createPatternNgrams(pleural_report, 4, pleural_keyword)
list(pleural_fourgramk.items())[:20]

[(('<s>', 'no', 'change', 'of'), 38),
 (('no', 'change', 'of', 'the'), 21),
 (('<s>', 'there', 'is', 'no'), 20),
 (('no', 'significant', 'change', 'of'), 18),
 (('there', 'is', 'no', 'change'), 17),
 (('is', 'no', 'change', 'of'), 17),
 (('<s>', 'no', 'significant', 'change'), 15),
 (('<s>', 'the', 'chest', 'shows'), 15),
 (('the', 'chest', 'shows', 'no'), 12),
 (('no', 'change', 'of', 'left'), 12),
 (('chest', 'shows', 'no', 'change'), 10),
 (('shows', 'no', 'change', 'of'), 9),
 (('left', 'pleural', 'fusion', '<\\s>'), 9),
 (('of', 'the', 'right', 'effusion'), 8),
 (('no', 'change', 'of', 'right'), 8),
 (('change', 'of', 'left', 'effusion'), 8),
 (('change', 'of', 'the', 'right'), 7),
 (('amount', 'of', 'right', 'effusion'), 7),
 (('change', 'of', 'right', 'pleural'), 7),
 (('of', 'left', 'effusion', '<\\s>'), 7)]

In [197]:
pleural_fivegramk = createPatternNgrams(pleural_report, 5, pleural_keyword)
list(pleural_fivegramk.items())[:20]

[(('<s>', 'there', 'is', 'no', 'change'), 17),
 (('there', 'is', 'no', 'change', 'of'), 17),
 (('<s>', 'no', 'significant', 'change', 'of'), 14),
 (('<s>', 'the', 'chest', 'shows', 'no'), 12),
 (('<s>', 'no', 'change', 'of', 'the'), 10),
 (('the', 'chest', 'shows', 'no', 'change'), 10),
 (('is', 'no', 'change', 'of', 'the'), 10),
 (('chest', 'shows', 'no', 'change', 'of'), 9),
 (('<s>', 'no', 'change', 'of', 'left'), 9),
 (('no', 'change', 'of', 'the', 'right'), 7),
 (('change', 'of', 'the', 'right', 'effusion'), 7),
 (('no', 'change', 'of', 'left', 'effusion'), 7),
 (('change', 'of', 'the', 'rt', 'effusion'), 6),
 (('<s>', 'minimal', 'left', 'pleural', 'fusion'), 6),
 (('change', 'of', 'left', 'effusion', '<\\s>'), 6),
 (('<s>', 'as', 'compared', 'with', 'prior'), 5),
 (('no', 'change', 'of', 'the', 'left'), 5),
 (('change', 'of', 'the', 'left', 'effusion'), 5),
 (('change', 'of', 'moderate', 'amount', 'of'), 5),
 (('no', 'change', 'of', 'right', 'pleural'), 5)]

In [198]:
pleural_sixgramk = createPatternNgrams(pleural_report, 6, pleural_keyword)
list(pleural_sixgramk.items())[:20]

[(('<s>', 'there', 'is', 'no', 'change', 'of'), 17),
 (('<s>', 'the', 'chest', 'shows', 'no', 'change'), 10),
 (('there', 'is', 'no', 'change', 'of', 'the'), 10),
 (('the', 'chest', 'shows', 'no', 'change', 'of'), 9),
 (('no', 'change', 'of', 'the', 'right', 'effusion'), 7),
 (('no', 'change', 'of', 'the', 'left', 'effusion'), 5),
 (('<s>', 'no', 'change', 'of', 'left', 'effusion'), 5),
 (('<s>', 'minimal', 'left', 'pleural', 'fusion', '<\\s>'), 5),
 (('no', 'change', 'of', 'the', 'rt', 'effusion'), 5),
 (('no', 'change', 'of', 'left', 'effusion', '<\\s>'), 5),
 (('<s>', 'as', 'compared', 'with', 'prior', 'study'), 4),
 (('as', 'compared', 'with', 'prior', 'study', 'on'), 4),
 (('no', 'significant', 'change', 'of', 'moderate', 'amount'), 4),
 (('change', 'of', 'the', 'right', 'effusion', '<\\s>'), 4),
 (('is', 'no', 'change', 'of', 'the', 'right'), 4),
 (('change', 'of', 'moderate', 'amount', 'of', 'left'), 4),
 (('change', 'of', 'the', 'rt', 'effusion', 'compared'), 4),
 (('of', 'the'

In [199]:
pleural_sevengramk = createPatternNgrams(pleural_report, 7, pleural_keyword)
list(pleural_sevengramk.items())[:20]

[(('<s>', 'there', 'is', 'no', 'change', 'of', 'the'), 10),
 (('<s>', 'the', 'chest', 'shows', 'no', 'change', 'of'), 9),
 (('<s>', 'as', 'compared', 'with', 'prior', 'study', 'on'), 4),
 (('no', 'change', 'of', 'the', 'right', 'effusion', '<\\s>'), 4),
 (('there', 'is', 'no', 'change', 'of', 'the', 'right'), 4),
 (('is', 'no', 'change', 'of', 'the', 'right', 'effusion'), 4),
 (('no', 'change', 'of', 'the', 'rt', 'effusion', 'compared'), 4),
 (('change', 'of', 'the', 'rt', 'effusion', 'compared', 'to'), 4),
 (('the', 'chest', 'shows', 'no', 'change', 'of', 'effusion'), 3),
 (('no', 'change', 'of', 'the', 'left', 'effusion', 'compared'), 3),
 (('change', 'of', 'the', 'left', 'effusion', 'compared', 'to'), 3),
 (('<s>', 'no', 'change', 'of', 'the', 'right', 'effusion'), 3),
 (('<s>', 'no', 'change', 'of', 'the', 'left', 'effusion'), 3),
 (('<s>', 'no', 'significant', 'change', 'of', 'moderate', 'amount'), 3),
 (('no', 'significant', 'change', 'of', 'moderate', 'amount', 'of'), 3),
 (('th

## Difference between Result 2 and 3

In [200]:
findDiffData('Pleural Effusion', pleural_data_old, pleural_data)

nan -1.0 ['<s> chest pa upright <\\s>', '<s> history of dyspnoea <\\s>', '<s> pace maker tip is located in the rv <\\s>', '<s> there is moderate cardiomegaly <\\s>', '<s> there is interstitial infiltration at the rll region seen <\\s>', '<s> old fx of the left 7 9th rib <\\s>', '<s> blunting of the right costophrenic angle probably fibrosis or adhesion or effusion <\\s>', '<s> osteoporosis <\\s>']
0.0 1.0 ['<s> cxr <\\s>', '<s> diffuse interstitial thickening of right lung is noted <\\s>', '<s> increased opacity at right apical lung could be infiltration or pleural thickening <\\s>', '<s> left lung is clear <\\s>', '<s> cardiomegaly <\\s>', '<s> bulging increased opacity at right paratracheal region could be svc and brachiocephalic vessel but mediastinal lesion cannot be r o <\\s>', '<s> nno pleural effusion <\\s>']
nan 1.0 ['<s> chest <\\s>', '<s> post sternotomy <\\s>', '<s> there is left pleural effusion <\\s>', '<s> normal heart size <\\s>', '<s> bones intact <\\s>']
nan 1.0 ['<s> 

In [201]:
findDiffData2('Pleural Effusion', pleural_data_old, pleural_data)

nan -1.0 ['<s> chest pa upright <\\s>', '<s> history of dyspnea <\\s>', '<s> pace maker tip is located in the rv <\\s>', '<s> there is moderate cardiomegaly <\\s>', '<s> there is interstitial infitlration at the rll region seen <\\s>', '<s> old fx of the left 7 9th rib <\\s>', '<s> blunting of the right costophenic angle probably fibrosis or adhesion or effusino <\\s>', '<s> osteoporosis <\\s>']
         ['<s> chest pa upright <\\s>', '<s> history of dyspnoea <\\s>', '<s> pace maker tip is located in the rv <\\s>', '<s> there is moderate cardiomegaly <\\s>', '<s> there is interstitial infiltration at the rll region seen <\\s>', '<s> old fx of the left 7 9th rib <\\s>', '<s> blunting of the right costophrenic angle probably fibrosis or adhesion or effusion <\\s>', '<s> osteoporosis <\\s>']
0.0 1.0 ['<s> cxr <\\s>', '<s> diffuse interstitial thickening of right lung is noted <\\s>', '<s> increased opacity at right apical lung could be infiltration or pleural thickening <\\s>', '<s> left 

# Atelectasis

In [202]:
# define keyword
# atelec_keyword = r'atelec|collap|volume'
atelec_keyword = r'atelec'

### Ground truth 0

Prepare Data

In [203]:
# choose reports
atelec_df_0 = selectData(atelec_data, 'Atelectasis', 0.0)
atelec_df_0

,Reports,Report List,Atelectasis BERT Labeler,Atelectasis Inspectra Labeler,Atelectasis Ground Truth,Atelectasis Key Detect
8,CXR \n\nHistory: none\nComparison: 31.5.2005\...,"['<s> cxr <\\s>', '<s> history none <\\s>', '<...",1,0,0.0,"<re.Match object; span=(4, 47), match='resolve..."
412,CXR PA upright\n\nHistory : ---\n\nComparison ...,"['<s> cxr pa upright <\\s>', '<s> history <\\s...",1,0,0.0,"<re.Match object; span=(4, 74), match='no demo..."
473,CXR PA upright\n\nMinimal right lower lung op...,"['<s> cxr pa upright <\\s>', '<s> minimal righ...",1,0,0.0,"<re.Match object; span=(4, 80), match='no demo..."
498,"CHEST FILM PA, UPRIGHT VIEW\n\nCOMPARISON: 07/...","['<s> chest film pa upright view <\\s>', '<s> ...",0,1,0.0,"<re.Match object; span=(4, 55), match='no inte..."
509,CXR\n Known case CA lung S/P CMT.\n Signific...,"['<s> cxr <\\s>', '<s> known case ca lung s p ...",0,1,0.0,"<re.Match object; span=(65, 131), match='no me..."
549,"CHEST, PA UPRIGHT\nAs comparison to prior CXR ...","['<s> chest pa upright <\\s>', '<s> as compari...",1,0,0.0,"<re.Match object; span=(81, 158), match='no sh..."
617,"CXR PA upright\n\nHistory : ESRD, HT, chronic...","['<s> cxr pa upright <\\s>', '<s> history esd ...",1,0,0.0,"<re.Match object; span=(4, 74), match='no demo..."
621,CXR PA upright\n\n Compared with previous che...,"['<s> cxr pa upright <\\s>', '<s> compared wit...",1,0,0.0,"<re.Match object; span=(20, 77), match='no mil..."
624,CXR PA upright\n\n\nAs compared with prior stu...,"['<s> cxr pa upright <\\s>', '<s> as compared ...",1,0,0.0,"<re.Match object; span=(4, 44), match='no demo..."
756,CXR PA upright\n\nHistory : CA breast. \n\nCo...,"['<s> cxr pa upright <\\s>', '<s> history ca b...",1,0,0.0,"<re.Match object; span=(4, 74), match='no demo..."


In [204]:
# create report list
atelec_report_0 = createReportList(atelec_df_0)
atelec_report_0[0]

['<s> cxr <\\s>',
 '<s> history none <\\s>',
 '<s> comparison 31.5.2005 <\\s>',
 '<s> no pulmonary infiltration <\\s>',
 '<s> no cardiomegaly <\\s>',
 '<s> no pleural effusion <\\s>',
 '<s> bony structures are intact <\\s>',
 '<s> resolved atelectasis of the rll region <\\s>',
 '<s> imp no detectable abnormality in this study <\\s>']

Find N-grams Pattern

In [205]:
atelec_bigram_0 = createPatternNgrams(atelec_report_0, 2, atelec_keyword)
list(atelec_bigram_0.items())[:20]

[(('<s>', 'no'), 23),
 (('atelectasis', '<\\s>'), 22),
 (('mass', 'or'), 17),
 (('or', 'atelectasis'), 17),
 (('no', 'demonstrable'), 16),
 (('pulmonary', 'infiltration'), 12),
 (('active', 'pulmonary'), 11),
 (('infiltration', 'mass'), 11),
 (('demonstrable', 'active'), 9),
 (('change', 'of'), 7),
 (('no', 'interval'), 5),
 (('interval', 'change'), 5),
 (('atelectasis', 'at'), 5),
 (('demonstrable', 'mass'), 5),
 (('is', 'noted'), 4),
 (('noted', '<\\s>'), 3),
 (('with', 'no'), 3),
 (('<s>', 'as'), 3),
 (('infiltration', 'or'), 3),
 (('segmental', 'atelectasis'), 3)]

In [206]:
atelec_trigram_0 = createPatternNgrams(atelec_report_0, 3, atelec_keyword)
list(atelec_trigram_0.items())[:20]

[(('mass', 'or', 'atelectasis'), 17),
 (('<s>', 'no', 'demonstrable'), 16),
 (('or', 'atelectasis', '<\\s>'), 16),
 (('active', 'pulmonary', 'infiltration'), 11),
 (('pulmonary', 'infiltration', 'mass'), 11),
 (('infiltration', 'mass', 'or'), 11),
 (('no', 'demonstrable', 'active'), 9),
 (('demonstrable', 'active', 'pulmonary'), 9),
 (('no', 'interval', 'change'), 5),
 (('interval', 'change', 'of'), 5),
 (('no', 'demonstrable', 'mass'), 5),
 (('demonstrable', 'mass', 'or'), 5),
 (('<s>', 'no', 'interval'), 3),
 (('is', 'noted', '<\\s>'), 3),
 (('<s>', 'resolved', 'atelectasis'), 2),
 (('no', 'demonstrable', 'other'), 2),
 (('demonstrable', 'other', 'active'), 2),
 (('other', 'active', 'pulmonary'), 2),
 (('change', 'of', 'rul'), 2),
 (('of', 'rul', 'atelectasis'), 2)]

### Ground truth 1

Prepare Data

In [207]:
# choose reports
atelec_df_1 = selectData(atelec_data, 'Atelectasis', 1.0)
atelec_df_1

,Reports,Report List,Atelectasis BERT Labeler,Atelectasis Inspectra Labeler,Atelectasis Ground Truth,Atelectasis Key Detect
4,CXR\n\nRUL ateletasis.\nFibrocalcific infiltra...,"['<s> cxr <\\s>', '<s> rul atelectasis <\\s>',...",1,0,1.0,"<re.Match object; span=(4, 24), match='rul ate..."
7,"Chest PA upright;\n known case CA lung, post C...","['<s> chest pa upright <\\s>', '<s> known case...",1,0,1.0,"<re.Match object; span=(62, 82), match='lll at..."
23,"CHEST.;\n\nKnoWn case of Ewing's sarcoma,post ...","['<s> chest <\\s>', '<s> known case of wings s...",1,0,1.0,"<re.Match object; span=(0, 20), match='<s> ate..."
25,CHEST\n\nAs compared with previous film on 20/...,"['<s> chest <\\s>', '<s> as compared with prev...",0,1,1.0,"<re.Match object; span=(0, 67), match='<s> ate..."
65,Chest PA.\n Ateletasis of RUL.\n right pleur...,"['<s> chest pa <\\s>', '<s> atelectasis of rul...",1,0,1.0,"<re.Match object; span=(0, 27), match='<s> ate..."
141,CXR\nAtectasis at RUL and opacity at RLL\nNorm...,"['<s> cxr <\\s>', '<s> atelectasis at rul and ...",1,0,1.0,"<re.Match object; span=(0, 46), match='<s> ate..."
176,CXR (PA)\n\nThe LLL opacity with sign of LLL v...,"['<s> cxr pa <\\s>', '<s> the lll opacity with...",1,0,1.0,"<re.Match object; span=(122, 142), match='lll ..."
185,Chest PA upruight \n\nHistory: RUL atelectasis...,"['<s> chest pa upright <\\s>', '<s> history ru...",1,0,1.0,"<re.Match object; span=(12, 32), match='rul at..."
186,CXR PA upright\n\nNo demonstrable active pulmo...,"['<s> cxr pa upright <\\s>', '<s> no demonstra...",0,1,1.0,"<re.Match object; span=(0, 64), match='<s> ate..."
317,CXR(PA upright)\n\n Hx of endobronchial tumor...,"['<s> cxr pa upright <\\s>', '<s> hx of endobr...",1,0,1.0,"<re.Match object; span=(43, 81), match='lul at..."


In [208]:
# create report list
atelec_report_1 = createReportList(atelec_df_1)
atelec_report_1[0]

['<s> cxr <\\s>',
 '<s> rul atelectasis <\\s>',
 '<s> fibrocalcific infiltration at both upper lobe and rll <\\s>',
 '<s> cardiomegaly <\\s>',
 '<s> no pleural effusion <\\s>',
 '<s> boat thorax is i8ntact <\\s>']


Find N-grams Pattern

In [209]:
atelec_bigram_1 = createPatternNgrams(atelec_report_1, 2, atelec_keyword)
list(atelec_bigram_1.items())[:20]

[(('<s>', 'atelectasis'), 27),
 (('atelectasis', '<\\s>'), 17),
 (('atelectasis', 'at'), 17),
 (('rul', 'atelectasis'), 12),
 (('lll', 'atelectasis'), 8),
 (('rul', '<\\s>'), 8),
 (('at', 'rul'), 8),
 (('of', 'rul'), 7),
 (('atelectasis', 'is'), 7),
 (('at', 'lll'), 6),
 (('<s>', 'rul'), 5),
 (('atelectasis', 'of'), 5),
 (('<s>', 'the'), 5),
 (('opacity', 'at'), 4),
 (('atelectasis', 'and'), 4),
 (('the', 'left'), 4),
 (('of', 'the'), 4),
 (('atelectasis', 'with'), 4),
 (('<s>', 'resolving'), 4),
 (('resolving', 'of'), 4)]

In [210]:
atelec_trigram_1 = createPatternNgrams(atelec_report_1, 3, atelec_keyword)
list(atelec_trigram_1.items())[:20]

[(('<s>', 'atelectasis', 'at'), 14),
 (('lll', 'atelectasis', '<\\s>'), 7),
 (('atelectasis', 'at', 'rul'), 6),
 (('<s>', 'rul', 'atelectasis'), 5),
 (('rul', 'atelectasis', '<\\s>'), 5),
 (('<s>', 'atelectasis', 'of'), 5),
 (('at', 'rul', '<\\s>'), 5),
 (('atelectasis', 'at', 'lll'), 5),
 (('resolving', 'of', 'rul'), 4),
 (('<s>', 'the', 'chest'), 4),
 (('the', 'chest', 'shows'), 4),
 (('<s>', 'atelectasis', '<\\s>'), 3),
 (('rul', 'atelectasis', 'is'), 3),
 (('of', 'rul', 'atelectasis'), 3),
 (('<s>', 'partial', 'resolved'), 3),
 (('rll', 'atelectasis', 'is'), 3),
 (('is', 'seen', '<\\s>'), 3),
 (('left', 'lung', '<\\s>'), 3),
 (('<s>', 'atelectasis', 'and'), 2),
 (('atelectasis', 'of', 'rul'), 2)]

## Ground truth -1

Prepare Data

In [211]:
# choose reports
atelec_df_u = selectData(atelec_data, 'Atelectasis', -1.0)
atelec_df_u

,Reports,Report List,Atelectasis BERT Labeler,Atelectasis Inspectra Labeler,Atelectasis Ground Truth,Atelectasis Key Detect
3,"CXR\n Compare with film on 6/5/05, less expan...","['<s> cxr <\\s>', '<s> compare with film on 6 ...",0,1,-1.0,"<re.Match object; span=(4, 46), match='no chan..."
5,CXR PA upright\nHistory of ovarian cyst. \n\nS...,"['<s> cxr pa upright <\\s>', '<s> history of o...",0,1,-1.0,"<re.Match object; span=(10, 56), match='plate ..."
6,CXR PA upright\n By compared with previous ch...,"['<s> cxr pa upright <\\s>', '<s> by compared ...",0,1,-1.0,"<re.Match object; span=(18, 72), match='some a..."
11,CHEST\n\nCompared to previous film on 28/07/20...,"['<s> chest <\\s>', '<s> compared to previous ...",0,1,-1.0,"<re.Match object; span=(4, 104), match='no sig..."
12,CHEST PA UPRIGHT\n\nHistory of lung tumor post...,"['<s> chest pa upright <\\s>', '<s> history of...",0,1,-1.0,"<re.Match object; span=(51, 78), match='plate ..."
...,...,...,...,...,...,...
2158,CXR PA upright\n\nHistory : RA with 1st hypot...,"['<s> cxr pa upright <\\s>', '<s> history ra w...",0,1,-1.0,"<re.Match object; span=(4, 62), match='no chan..."
2159,CXR(PA)\n\n Compare to CXR on 15/5/2015 and 1...,"['<s> cxr pa <\\s>', '<s> compare to cxr on 15...",0,1,-1.0,"<re.Match object; span=(4, 72), match='no chan..."
2162,Chest PA upright\n \n No pulmonary infiltrat...,"['<s> chest pa upright <\\s>', '<s> no pulmona...",1,0,-1.0,"<re.Match object; span=(4, 73), match='plate a..."
2163,CHEST FILM\n\nNo pulmonary infiltration is see...,"['<s> chest film <\\s>', '<s> no pulmonary inf...",1,0,-1.0,"<re.Match object; span=(4, 65), match='plate l..."


In [212]:
# create report list
atelec_report_u = createReportList(atelec_df_u)
atelec_report_u[0]

['<s> cxr <\\s>',
 '<s> compare with film on 6 5 05 less expansion of bilateral lungs are noted and possibly causing more prominent of interstitial thickening at lll <\\s>',
 '<s> no change of plate atelectasis at rll <\\s>',
 '<s> no cardiomegaly <\\s>',
 '<s> mildly prominent aorta is noted <\\s>',
 '<s> no pleural effusion <\\s>',
 '<s> intact bony thorax <\\s>']

Find N-grams Pattern

In [213]:
atelec_bigram_u = createPatternNgrams(atelec_report_u, 2, atelec_keyword)
list(atelec_bigram_u.items())[:20]

[(('atelectasis', 'at'), 708),
 (('change', 'of'), 538),
 (('<s>', 'no'), 490),
 (('plate', 'atelectasis'), 489),
 (('no', 'change'), 414),
 (('<s>', 'plate'), 234),
 (('at', 'left'), 229),
 (('of', 'plate'), 215),
 (('at', 'lll'), 209),
 (('at', 'right'), 176),
 (('no', 'significant'), 159),
 (('lung', '<\\s>'), 159),
 (('significant', 'change'), 132),
 (('plate', 'like'), 127),
 (('like', 'atelectasis'), 126),
 (('lll', '<\\s>'), 120),
 (('lower', 'lung'), 118),
 (('of', 'atelectasis'), 115),
 (('changed', 'of'), 111),
 (('atelectasis', '<\\s>'), 107)]

In [214]:
atelec_trigram_u = createPatternNgrams(atelec_report_u, 3, atelec_keyword)
list(atelec_trigram_u.items())[:20]

[(('plate', 'atelectasis', 'at'), 406),
 (('no', 'change', 'of'), 402),
 (('<s>', 'no', 'change'), 281),
 (('atelectasis', 'at', 'left'), 207),
 (('of', 'plate', 'atelectasis'), 198),
 (('atelectasis', 'at', 'lll'), 185),
 (('change', 'of', 'plate'), 158),
 (('<s>', 'plate', 'atelectasis'), 134),
 (('no', 'significant', 'change'), 132),
 (('significant', 'change', 'of'), 127),
 (('plate', 'like', 'atelectasis'), 126),
 (('atelectasis', 'at', 'right'), 125),
 (('like', 'atelectasis', 'at'), 112),
 (('<s>', 'no', 'significant'), 111),
 (('at', 'lll', '<\\s>'), 104),
 (('change', 'of', 'atelectasis'), 95),
 (('no', 'changed', 'of'), 88),
 (('is', 'noted', '<\\s>'), 86),
 (('<s>', 'plate', 'like'), 80),
 (('at', 'lll', 'is'), 75)]

## Unlabel

Prepare Data

In [215]:
# choose reports
atelec_df = selectData(atelec_data, 'Atelectasis', None)
atelec_df

,Reports,Report List,Atelectasis BERT Labeler,Atelectasis Inspectra Labeler,Atelectasis Ground Truth,Atelectasis Key Detect
0,CXR\n Suboptimal inspiration is noted.\n Inc...,"['<s> cxr <\\s>', '<s> suboptimal inspiration ...",1,0,NaN,NaN
1,Chest film\nReticulonodular infiltration both ...,"['<s> chest film <\\s>', '<s> reticulonodular ...",1,0,NaN,NaN
2,CXR PA upright\nFibropathy and atelectasis at ...,"['<s> cxr pa upright <\\s>', '<s> fibropathy a...",0,1,NaN,NaN
9,CHEST\n\nPlate atelecsis at Rt lower lobe.\nFi...,"['<s> chest <\\s>', '<s> plate telesis at rt l...",1,0,NaN,NaN
10,CXR\n\nCardiomegaly with increase pulmonary va...,"['<s> cxr <\\s>', '<s> cardiomegaly with incre...",1,0,NaN,NaN
...,...,...,...,...,...,...
2164,Chest PA upright\n Compare to prior study date...,"['<s> chest pa upright <\\s>', '<s> compare to...",1,0,NaN,NaN
2165,CXR PA UPRIGHT\nFINDINGS:\nIncrease in amount ...,"['<s> cxr pa upright <\\s>', '<s> findings <\\...",1,0,NaN,NaN
2166,CXR PA UPRIGHT\n\nFINDINGS:\nFurther increase ...,"['<s> cxr pa upright <\\s>', '<s> findings <\\...",1,0,NaN,NaN
2168,CXR PA\n\nCompared study: 18/04/2018\nSuspicio...,"['<s> cxr pa <\\s>', '<s> compared study 18 04...",1,0,NaN,NaN


In [216]:
# create report list
atelec_report = createReportList(atelec_df)
atelec_report[0]

['<s> cxr <\\s>',
 '<s> suboptimal inspiration is noted <\\s>',
 '<s> increased lung marking at both basal lungs are noted and possibly due to collapse lung markings from not full inspiration <\\s>',
 '<s> no definite lung filtration or nodule is seen <\\s>',
 '<s> heart size cannot be evaluate <\\s>']


Find N-grams Pattern

In [217]:
atelec_bigram = createPatternNgrams(atelec_report, 2)
list(atelec_bigram.items())[:20]

[(('<s>', 'no'), 1077),
 (('noted', '<\\s>'), 666),
 (('<s>', 'the'), 640),
 (('is', 'seen'), 580),
 (('is', 'noted'), 578),
 (('seen', '<\\s>'), 576),
 (('pleural', 'effusion'), 436),
 (('change', 'of'), 376),
 (('clear', '<\\s>'), 351),
 (('pulmonary', 'infiltration'), 323),
 (('both', 'costophrenic'), 313),
 (('degenerative', 'change'), 268),
 (('effusion', '<\\s>'), 263),
 (('cardiomegaly', 'is'), 260),
 (('<s>', 'findings'), 259),
 (('costophrenic', 'angles'), 253),
 (('are', 'clear'), 250),
 (('left', 'lung'), 246),
 (('angles', 'are'), 244),
 (('<s>', 'both'), 239)]

In [218]:
atelec_trigram = createPatternNgrams(atelec_report, 3)
list(atelec_trigram.items())[:20]

[(('is', 'noted', '<\\s>'), 509),
 (('is', 'seen', '<\\s>'), 504),
 (('are', 'clear', '<\\s>'), 248),
 (('costophrenic', 'angles', 'are'), 244),
 (('pleural', 'effusion', '<\\s>'), 238),
 (('both', 'costophrenic', 'angles'), 230),
 (('<s>', 'no', 'cardiomegaly'), 226),
 (('<s>', 'both', 'costophrenic'), 185),
 (('cardiomegaly', 'is', 'seen'), 162),
 (('pulmonary', 'infiltration', 'or'), 157),
 (('calcified', 'aortic', 'knob'), 155),
 (('degenerative', 'change', 'of'), 150),
 (('no', 'cardiomegaly', 'is'), 149),
 (('<s>', 'degenerative', 'change'), 144),
 (('is', 'intact', '<\\s>'), 139),
 (('bony', 'thorax', 'is'), 132),
 (('<s>', 'there', 'is'), 131),
 (('<s>', 'calcified', 'aortic'), 131),
 (('blunting', 'of', 'the'), 131),
 (('<s>', 'no', 'blunting'), 129)]

In [219]:
atelec_fourgram = createPatternNgrams(atelec_report, 4)
list(atelec_fourgram.items())[:20]

[(('both', 'costophrenic', 'angles', 'are'), 224),
 (('cardiomegaly', 'is', 'seen', '<\\s>'), 161),
 (('<s>', 'no', 'cardiomegaly', 'is'), 149),
 (('<s>', 'degenerative', 'change', 'of'), 135),
 (('<s>', 'calcified', 'aortic', 'knob'), 129),
 (('<s>', 'no', 'blunting', 'of'), 129),
 (('<s>', 'no', 'pleural', 'effusion'), 125),
 (('costophrenic', 'angles', 'are', 'clear'), 125),
 (('angles', 'are', 'clear', '<\\s>'), 124),
 (('no', 'blunting', 'of', 'the'), 123),
 (('bony', 'thorax', 'is', 'intact'), 121),
 (('thorax', 'is', 'intact', '<\\s>'), 121),
 (('visualized', 'bony', 'structures', 'are'), 118),
 (('<s>', 'the', 'visualized', 'bony'), 116),
 (('the', 'visualized', 'bony', 'structures'), 115),
 (('calcified', 'aortic', 'knob', 'is'), 111),
 (('is', 'not', 'enlarged', '<\\s>'), 109),
 (('<s>', 'both', 'costophrenic', 'angles'), 107),
 (('pulmonary', 'infiltration', 'or', 'nodule'), 102),
 (('heart', 'is', 'not', 'enlarged'), 99)]

In [220]:
atelec_bigramk = createPatternNgrams(atelec_report, 2, atelec_keyword)
list(atelec_bigramk.items())[:20]

[(('atelectasis', '<\\s>'), 23),
 (('and', 'atelectasis'), 15),
 (('atelectasis', 'at'), 15),
 (('of', 'right'), 14),
 (('at', 'rul'), 12),
 (('right', 'lung'), 10),
 (('pleural', 'effusion'), 9),
 (('<s>', 'there'), 9),
 (('at', 'right'), 8),
 (('there', 'is'), 8),
 (('left', 'lung'), 7),
 (('elevation', 'of'), 6),
 (('rml', 'atelectasis'), 6),
 (('atelectasis', 'is'), 6),
 (('at', 'lul'), 5),
 (('is', 'noted'), 5),
 (('right', 'pleural'), 5),
 (('atelectasis', 'of'), 5),
 (('lung', '<\\s>'), 5),
 (('no', 'change'), 5)]

In [221]:
atelec_trigramk = createPatternNgrams(atelec_report, 3, atelec_keyword)
list(atelec_trigramk.items())[:20]

[(('and', 'atelectasis', 'at'), 10),
 (('atelectasis', 'at', 'rul'), 8),
 (('<s>', 'there', 'is'), 8),
 (('right', 'pleural', 'effusion'), 5),
 (('of', 'right', 'lung'), 5),
 (('no', 'change', 'of'), 5),
 (('at', 'rul', '<\\s>'), 4),
 (('pleural', 'effusion', 'with'), 4),
 (('<s>', 'no', 'change'), 4),
 (('fibrosis', 'and', 'atelectasis'), 3),
 (('at', 'rul', 'and'), 3),
 (('at', 'lul', '<\\s>'), 3),
 (('<s>', 'pulmonary', 'infiltration'), 3),
 (('elevation', 'of', 'right'), 3),
 (('atelectasis', 'of', 'right'), 3),
 (('right', 'lung', '<\\s>'), 3),
 (('compressive', 'atelectasis', '<\\s>'), 3),
 (('atelectasis', 'at', 'right'), 3),
 (('right', 'upper', 'lobe'), 3),
 (('lul', 'infiltration', 'and'), 3)]

In [222]:
atelec_fourgramk = createPatternNgrams(atelec_report, 4, atelec_keyword)
list(atelec_fourgramk.items())[:20]

[(('and', 'atelectasis', 'at', 'rul'), 6),
 (('atelectasis', 'at', 'rul', '<\\s>'), 4),
 (('<s>', 'no', 'change', 'of'), 4),
 (('atelectasis', 'of', 'right', 'lung'), 3),
 (('lul', 'infiltration', 'and', 'atelectasis'), 3),
 (('infiltration', 'and', 'atelectasis', '<\\s>'), 3),
 (('mass', 'and', 'atelectasis', 'at'), 2),
 (('<s>', 'fibrosis', 'and', 'atelectasis'), 2),
 (('fibrosis', 'and', 'atelectasis', 'at'), 2),
 (('atelectasis', 'at', 'rul', 'and'), 2),
 (('right', 'pleural', 'effusion', '<\\s>'), 2),
 (('right', 'lower', 'lobe', 'atelectasis'), 2),
 (('atelectasis', 'are', 'noted', '<\\s>'), 2),
 (('<s>', 'fibroinfiltration', 'and', 'atelectasis'), 2),
 (('of', 'right', 'lung', '<\\s>'), 2),
 (('<s>', 'there', 'is', 'elevation'), 2),
 (('there', 'is', 'elevation', 'of'), 2),
 (('is', 'elevation', 'of', 'the'), 2),
 (('elevation', 'of', 'the', 'left'), 2),
 (('of', 'the', 'left', 'dome'), 2)]

In [223]:
atelec_fivegramk = createPatternNgrams(atelec_report, 5, atelec_keyword)
list(atelec_fivegramk.items())[:20]

[(('and', 'atelectasis', 'at', 'rul', '<\\s>'), 4),
 (('lul', 'infiltration', 'and', 'atelectasis', '<\\s>'), 3),
 (('mass', 'and', 'atelectasis', 'at', 'rul'), 2),
 (('<s>', 'fibrosis', 'and', 'atelectasis', 'at'), 2),
 (('and', 'atelectasis', 'at', 'rul', 'and'), 2),
 (('atelectasis', 'of', 'right', 'lung', '<\\s>'), 2),
 (('<s>', 'there', 'is', 'elevation', 'of'), 2),
 (('there', 'is', 'elevation', 'of', 'the'), 2),
 (('is', 'elevation', 'of', 'the', 'left'), 2),
 (('elevation', 'of', 'the', 'left', 'dome'), 2),
 (('of', 'the', 'left', 'dome', 'of'), 2),
 (('the', 'left', 'dome', 'of', 'diaphragm'), 2),
 (('platatelectasis', 'at', 'right', 'lower', 'lung'), 2),
 (('at', 'right', 'lower', 'lung', 'with'), 2),
 (('right', 'lower', 'lung', 'with', 'minimal'), 2),
 (('lower', 'lung', 'with', 'minimal', 'blunting'), 2),
 (('lung', 'with', 'minimal', 'blunting', 'of'), 2),
 (('with', 'minimal', 'blunting', 'of', 'right'), 2),
 (('minimal', 'blunting', 'of', 'right', 'costophrenic'), 2),
 

In [224]:
atelec_sixgramk = createPatternNgrams(atelec_report, 6, atelec_keyword)
list(atelec_sixgramk.items())[:20]

[(('mass', 'and', 'atelectasis', 'at', 'rul', '<\\s>'), 2),
 (('<s>', 'there', 'is', 'elevation', 'of', 'the'), 2),
 (('there', 'is', 'elevation', 'of', 'the', 'left'), 2),
 (('is', 'elevation', 'of', 'the', 'left', 'dome'), 2),
 (('elevation', 'of', 'the', 'left', 'dome', 'of'), 2),
 (('of', 'the', 'left', 'dome', 'of', 'diaphragm'), 2),
 (('platatelectasis', 'at', 'right', 'lower', 'lung', 'with'), 2),
 (('at', 'right', 'lower', 'lung', 'with', 'minimal'), 2),
 (('right', 'lower', 'lung', 'with', 'minimal', 'blunting'), 2),
 (('lower', 'lung', 'with', 'minimal', 'blunting', 'of'), 2),
 (('lung', 'with', 'minimal', 'blunting', 'of', 'right'), 2),
 (('with', 'minimal', 'blunting', 'of', 'right', 'costophrenic'), 2),
 (('minimal', 'blunting', 'of', 'right', 'costophrenic', 'angle'), 2),
 (('blunting', 'of', 'right', 'costophrenic', 'angle', 'from'), 2),
 (('of', 'right', 'costophrenic', 'angle', 'from', 'minimal'), 2),
 (('right', 'costophrenic', 'angle', 'from', 'minimal', 'pleural'), 

## Difference between Result 2 and 3

In [225]:
findDiffData('Atelectasis', atelec_data_old, atelec_data)

nan 0.0 ['<s> cxr <\\s>', '<s> history none <\\s>', '<s> comparison 31.5.2005 <\\s>', '<s> no pulmonary infiltration <\\s>', '<s> no cardiomegaly <\\s>', '<s> no pleural effusion <\\s>', '<s> bony structures are intact <\\s>', '<s> resolved atelectasis of the rll region <\\s>', '<s> imp no detectable abnormality in this study <\\s>']
-1.0 nan ['<s> chest <\\s>', '<s> plate telesis at rt lower lobe <\\s>', '<s> fibrotic change at lll <\\s>', '<s> no cardiomegaly or pleural effusion <\\s>']
0.0 -1.0 ['<s> chest <\\s>', '<s> compared with 6 5 2008 no change of rml infiltration and atelectasis <\\s>', '<s> normal cardiothoracic ratio <\\s>']
nan -1.0 ['<s> chest pa <\\s>', '<s> compare with 21 04 2010 <\\s>', '<s> unchanged of near total haziness at right lung with shifted of mediastinum to the right represented right pleural effusion with right lung atelectasis <\\s>', '<s> nodular infiltration at lul is seen <\\s>', '<s> no cardiomegaly is detected <\\s>']
nan 1.0 ['<s> cxr <\\s>', '<s> 

In [226]:
findDiffData2('Atelectasis', atelec_data_old, atelec_data)

nan 0.0 ['<s> cxr <\\s>', '<s> history none <\\s>', '<s> comparison 31.5.2005 <\\s>', '<s> no pulmonary infiltration <\\s>', '<s> no cardiomegaly <\\s>', '<s> no pleural effusion <\\s>', '<s> bony structures are intact <\\s>', '<s> resolved atelectasis of the rll region <\\s>', '<s> imp no detectable abnormality in this study <\\s>']
         ['<s> cxr <\\s>', '<s> history none <\\s>', '<s> comparison 31.5.2005 <\\s>', '<s> no pulmonary infiltration <\\s>', '<s> no cardiomegaly <\\s>', '<s> no pleural effusion <\\s>', '<s> bony structures are intact <\\s>', '<s> resolved atelectasis of the rll region <\\s>', '<s> imp no detectable abnormality in this study <\\s>']
-1.0 nan ['<s> chest <\\s>', '<s> plate atelecsis at rt lower lobe <\\s>', '<s> fibrotic change at lll <\\s>', '<s> no cardiomegaly or pleural effusion <\\s>']
         ['<s> chest <\\s>', '<s> plate telesis at rt lower lobe <\\s>', '<s> fibrotic change at lll <\\s>', '<s> no cardiomegaly or pleural effusion <\\s>']
0.0 -1.0 

# Lung Lesion

In [227]:
# define keyword
lesion_keyword = r'nodule|mass'

## Ground truth 0

Prepare Data

In [228]:
# choose reports
lesion_df_0 = selectData(lesion_data, 'Lung Lesion', 0.0)
lesion_df_0

,Reports,Report List,Lung Lesion BERT Labeler,Lung Lesion Inspectra Labeler,Lung Lesion Ground Truth,Lung Lesion Key Detect
8,CXR\n\nHistory of goiter\n\nMild hyperinflatio...,"['<s> cxr <\\s>', '<s> history of gaiter <\\s>...",1,0,0.0,"<re.Match object; span=(4, 23), match='no pulm..."
70,Chest PA upright\n Known case CA breast S/P CM...,"['<s> chest pa upright <\\s>', '<s> known case...",1,0,0.0,"<re.Match object; span=(4, 23), match='no pulm..."
72,CXR (PA upright)\nHx: Lt.parotidf tumor. Preo...,"['<s> cxr pa upright <\\s>', '<s> hx lt.paroti...",1,0,0.0,"<re.Match object; span=(4, 49), match='no defi..."
100,Chest film\nReticulonodular infiltration both ...,"['<s> chest film <\\s>', '<s> reticulonodular ...",1,0,0.0,"<re.Match object; span=(4, 20), match='no defi..."
119,Chest PA upright;\n compared to filmon 13/5/20...,"['<s> chest pa upright <\\s>', '<s> compared t...",0,1,0.0,"<re.Match object; span=(4, 33), match='no new ..."
...,...,...,...,...,...,...
34201,CXR PA upright\n\n There is no definite pulmo...,"['<s> cxr pa upright <\\s>', '<s> there is no ...",0,1,0.0,"<re.Match object; span=(13, 57), match='no def..."
34227,CXR PA upright\n\nHistory : ESRD \n\nCompari...,"['<s> cxr pa upright <\\s>', '<s> history esd ...",0,1,0.0,"<re.Match object; span=(4, 54), match='no demo..."
34267,CXR PA UPRIGHT \n\nFINDINGS:\n No pulmonary i...,"['<s> cxr pa upright <\\s>', '<s> findings <\\...",0,1,0.0,"<re.Match object; span=(4, 39), match='no pulm..."
34294,CXR\n\nCompared to prior film on 9 december 17...,"['<s> cxr <\\s>', '<s> compared to prior film ...",0,1,0.0,"<re.Match object; span=(57, 85), match='no def..."


In [229]:
# create report list
lesion_report_0 = createReportList(lesion_df_0)
lesion_report_0[0]

['<s> cxr <\\s>',
 '<s> history of gaiter <\\s>',
 '<s> mild hyperinflation is noted bilaterally <\\s>',
 '<s> blunting of right costophrenic angle could be small pleural effusion or pleural thickening <\\s>',
 '<s> calcification at right lower lobe close to lateral chest wall <\\s>',
 '<s> interstitial thickening both lungs <\\s>',
 '<s> no pulmonary nodule is demonstrated <\\s>']

Find N-grams Pattern

In [230]:
lesion_bigram_0 = createPatternNgrams(lesion_report_0, 2, lesion_keyword)
list(lesion_bigram_0.items())[:20]

[(('<s>', 'no'), 840),
 (('pulmonary', 'infiltration'), 724),
 (('infiltration', 'or'), 465),
 (('or', 'nodule'), 398),
 (('active', 'pulmonary'), 359),
 (('is', 'noted'), 310),
 (('no', 'demonstrable'), 306),
 (('noted', '<\\s>'), 298),
 (('nodule', 'is'), 294),
 (('mass', 'or'), 293),
 (('or', 'atelectasis'), 283),
 (('atelectasis', '<\\s>'), 280),
 (('infiltration', 'mass'), 261),
 (('demonstrable', 'active'), 247),
 (('no', 'detectable'), 217),
 (('no', 'definite'), 203),
 (('nodule', 'at'), 189),
 (('nodule', '<\\s>'), 185),
 (('detectable', 'pulmonary'), 164),
 (('change', 'of'), 161)]

In [231]:
lesion_trigram_0 = createPatternNgrams(lesion_report_0, 3, lesion_keyword)
list(lesion_trigram_0.items())[:20]

[(('pulmonary', 'infiltration', 'or'), 405),
 (('infiltration', 'or', 'nodule'), 390),
 (('active', 'pulmonary', 'infiltration'), 356),
 (('<s>', 'no', 'demonstrable'), 305),
 (('is', 'noted', '<\\s>'), 283),
 (('mass', 'or', 'atelectasis'), 283),
 (('or', 'atelectasis', '<\\s>'), 279),
 (('pulmonary', 'infiltration', 'mass'), 261),
 (('infiltration', 'mass', 'or'), 248),
 (('demonstrable', 'active', 'pulmonary'), 247),
 (('no', 'demonstrable', 'active'), 246),
 (('or', 'nodule', 'is'), 236),
 (('nodule', 'is', 'noted'), 188),
 (('no', 'detectable', 'pulmonary'), 163),
 (('detectable', 'pulmonary', 'infiltration'), 151),
 (('<s>', 'findings', 'no'), 149),
 (('findings', 'no', 'detectable'), 147),
 (('no', 'definite', 'pulmonary'), 118),
 (('or', 'nodule', '<\\s>'), 114),
 (('is', 'seen', '<\\s>'), 102)]

## Ground truth 1

Prepare Data

In [232]:
# choose reports
lesion_df_1 = selectData(lesion_data, 'Lung Lesion', 1.0)
lesion_df_1

,Reports,Report List,Lung Lesion BERT Labeler,Lung Lesion Inspectra Labeler,Lung Lesion Ground Truth,Lung Lesion Key Detect
14,CXR PA upright\n The chest shows multiple nod...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",1,0,1.0,"<re.Match object; span=(0, 14), match='<s> the..."
36,PA CXR\nClinical history: pre-op on thyroidal...,"['<s> pa cxr <\\s>', '<s> clinical history pre...",1,0,1.0,"<re.Match object; span=(0, 43), match='<s> cli..."
37,CXR\nClinical history: pre-op tonsillar mass\n...,"['<s> cxr <\\s>', '<s> clinical history pre op...",1,0,1.0,"<re.Match object; span=(0, 42), match='<s> cli..."
116,CXR (PA upright)\n\nHistory: thyroid nodule\n...,"['<s> cxr pa upright <\\s>', '<s> history thyr...",1,0,1.0,"<re.Match object; span=(0, 26), match='<s> his..."
120,CXR (PA upright)\nHx: Lt.lateral neck mass.\n...,"['<s> cxr pa upright <\\s>', '<s> hx lt.latera...",1,0,1.0,"<re.Match object; span=(0, 27), match='<s> hx ..."
...,...,...,...,...,...,...
33911,CXR\n\nNumerous vary size pulmonary nodules sc...,"['<s> cxr <\\s>', '<s> numerous vary size pulm...",1,0,1.0,"<re.Match object; span=(0, 39), match='<s> num..."
33979,"Chest PA upright \n\nMild cardiomegaly, CT rat...","['<s> chest pa upright <\\s>', '<s> mild cardi...",0,1,1.0,"<re.Match object; span=(0, 23), match='<s> ret..."
34106,CXR PA\n\nCompared with previous study: 8/3/20...,"['<s> cxr pa <\\s>', '<s> compared with previo...",0,1,1.0,"<re.Match object; span=(0, 45), match='<s> tot..."
34178,CXR (PA)\n\nThere is patchy opacity at left pa...,"['<s> cxr pa <\\s>', '<s> there is patchy opac...",1,0,1.0,"<re.Match object; span=(28, 141), match='at le..."


In [233]:
# create report list
lesion_report_1 = createReportList(lesion_df_1)
lesion_report_1[0]

['<s> cxr pa upright <\\s>',
 '<s> the chest shows multiple nodular infiltrations at both lungs metastasis should be considered <\\s>',
 '<s> the nodules are not seen on previous study <\\s>',
 '<s> the heart is not enlarged <\\s>',
 '<s> both costophrenic angles are clear <\\s>',
 '<s> the bony thorax is intact <\\s>']

Find N-grams Pattern

In [234]:
lesion_bigram_1 = createPatternNgrams(lesion_report_1, 2, lesion_keyword)
list(lesion_bigram_1.items())[:20]

[(('<s>', 'history'), 112),
 (('mass', '<\\s>'), 90),
 (('nodule', '<\\s>'), 74),
 (('nodule', 'at'), 74),
 (('<s>', 'no'), 73),
 (('pulmonary', 'infiltration'), 50),
 (('pulmonary', 'nodule'), 50),
 (('pleural', 'effusion'), 44),
 (('mass', 'at'), 43),
 (('at', 'left'), 43),
 (('<s>', 'the'), 40),
 (('at', 'right'), 40),
 (('is', 'seen'), 40),
 (('at', 'rul'), 38),
 (('history', 'of'), 37),
 (('<s>', 'there'), 37),
 (('there', 'is'), 36),
 (('lung', 'nodule'), 35),
 (('active', 'pulmonary'), 33),
 (('thyroid', 'nodule'), 32)]

In [235]:
lesion_trigram_1 = createPatternNgrams(lesion_report_1, 3, lesion_keyword)
list(lesion_trigram_1.items())[:20]

[(('<s>', 'there', 'is'), 32),
 (('active', 'pulmonary', 'infiltration'), 32),
 (('is', 'noted', '<\\s>'), 26),
 (('pleural', 'effusion', '<\\s>'), 26),
 (('thyroid', 'nodule', '<\\s>'), 25),
 (('<s>', 'no', 'demonstrable'), 25),
 (('<s>', 'history', 'of'), 23),
 (('right', 'pleural', 'effusion'), 22),
 (('nodule', 'at', 'rul'), 20),
 (('pulmonary', 'infiltration', 'mass'), 20),
 (('infiltration', 'mass', 'or'), 20),
 (('mass', 'or', 'atelectasis'), 20),
 (('<s>', 'clinical', 'history'), 19),
 (('pulmonary', 'nodule', 'at'), 18),
 (('pulmonary', 'infiltration', 'or'), 18),
 (('is', 'seen', '<\\s>'), 18),
 (('no', 'demonstrable', 'active'), 18),
 (('demonstrable', 'active', 'pulmonary'), 18),
 (('or', 'atelectasis', '<\\s>'), 18),
 (('massive', 'right', 'pleural'), 17)]

## Ground truth -1

Prepare Data

In [236]:
# choose reports
lesion_df_u = selectData(lesion_data, 'Lung Lesion', -1.0)
lesion_df_u

,Reports,Report List,Lung Lesion BERT Labeler,Lung Lesion Inspectra Labeler,Lung Lesion Ground Truth,Lung Lesion Key Detect
7,CXR\n\n No significant change of a RUL nodu...,"['<s> cxr <\\s>', '<s> no significant change o...",0,1,-1.0,"<re.Match object; span=(4, 41), match='no sign..."
13,Chest;\n\n Small calcified nodule at right lo...,"['<s> chest <\\s>', '<s> small calcified nodul...",0,1,-1.0,"<re.Match object; span=(4, 26), match='small c..."
23,CXR\n\nHistory of CA rectum\n\nAs compared to ...,"['<s> cxr <\\s>', '<s> history of ca rectum <\...",0,1,-1.0,"<re.Match object; span=(59, 98), match='no cha..."
25,CXR PA upright\n The chest shows multiple cal...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,-1.0,"<re.Match object; span=(20, 44), match='multip..."
31,"CXR\n Compare with film on 21/4/05, no change...","['<s> cxr <\\s>', '<s> compare with film on 21...",1,0,-1.0,"<re.Match object; span=(33, 85), match='no cha..."
...,...,...,...,...,...,...
34319,CXR PA upright\n\nComparison to prior radiogra...,"['<s> cxr pa upright <\\s>', '<s> comparison t...",0,1,-1.0,"<re.Match object; span=(26, 41), match='multip..."
34328,Chest PA upright\n\n Overall no significant c...,"['<s> chest pa upright <\\s>', '<s> overall no...",0,1,-1.0,"<re.Match object; span=(12, 68), match='no sig..."
34331,CXR PA UPRIGHT\n\nThere is a small calcified n...,"['<s> cxr pa upright <\\s>', '<s> there is a s...",0,1,-1.0,"<re.Match object; span=(15, 37), match='small ..."
34336,CXR PA upright\n\n The chest shows no change ...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,-1.0,"<re.Match object; span=(20, 52), match='no cha..."


In [237]:
# create report list
lesion_report_u = createReportList(lesion_df_u)
lesion_report_u[0]

['<s> cxr <\\s>',
 '<s> no significant change of a rul nodule as compare with film on 5 2 05 <\\s>',
 '<s> rest lungs are clear <\\s>',
 '<s> no pleural effusion <\\s>',
 '<s> cardiomegaly with markedly dilated and calcified aorta suggested aneurysm <\\s>']

Find N-grams Pattern

In [238]:
lesion_bigram_u = createPatternNgrams(lesion_report_u, 2, lesion_keyword)
list(lesion_bigram_u.items())[:20]

[(('change', 'of'), 5856),
 (('nodule', 'at'), 3632),
 (('<s>', 'no'), 3623),
 (('no', 'change'), 3359),
 (('no', 'significant'), 2512),
 (('significant', 'change'), 2302),
 (('at', 'right'), 2188),
 (('calcified', 'nodule'), 2187),
 (('nodules', 'at'), 2018),
 (('<s>', 'the'), 1760),
 (('the', 'chest'), 1560),
 (('chest', 'shows'), 1556),
 (('shows', 'no'), 1482),
 (('at', 'left'), 1367),
 (('at', 'both'), 1232),
 (('lung', '<\\s>'), 1192),
 (('at', 'rul'), 1187),
 (('of', 'multiple'), 1123),
 (('calcific', 'nodule'), 1112),
 (('of', 'a'), 1067)]

In [239]:
lesion_trigram_u = createPatternNgrams(lesion_report_u, 3, lesion_keyword)
list(lesion_trigram_u.items())[:20]

[(('no', 'change', 'of'), 3297),
 (('no', 'significant', 'change'), 2299),
 (('significant', 'change', 'of'), 2275),
 (('calcified', 'nodule', 'at'), 1742),
 (('<s>', 'no', 'significant'), 1705),
 (('<s>', 'the', 'chest'), 1558),
 (('the', 'chest', 'shows'), 1556),
 (('chest', 'shows', 'no'), 1453),
 (('shows', 'no', 'change'), 1347),
 (('<s>', 'no', 'change'), 1215),
 (('change', 'of', 'multiple'), 1031),
 (('nodule', 'at', 'right'), 942),
 (('change', 'of', 'a'), 894),
 (('nodules', 'at', 'both'), 885),
 (('calcific', 'nodule', 'at'), 793),
 (('both', 'lungs', '<\\s>'), 715),
 (('at', 'both', 'lungs'), 713),
 (('small', 'calcified', 'nodule'), 669),
 (('nodule', 'at', 'rul'), 667),
 (('<s>', 'there', 'is'), 654)]

## Unlabel

Prepare Data

In [240]:
# choose reports
lesion_df = selectData(lesion_data, 'Lung Lesion', None)
lesion_df

,Reports,Report List,Lung Lesion BERT Labeler,Lung Lesion Inspectra Labeler,Lung Lesion Ground Truth,Lung Lesion Key Detect
0,CXR (PA upright)\n\nReticulonodular infiltrati...,"['<s> cxr pa upright <\\s>', '<s> reticulonodu...",1,0,NaN,NaN
1,Chest:-\nPa chest study reveals fibronodular i...,"['<s> chest <\\s>', '<s> pa chest study reveal...",1,0,NaN,NaN
2,CHEST :\nP.A. upright view .\nFibronodular inf...,"['<s> chest <\\s>', '<s> p.a <\\s>', '<s> upri...",1,0,NaN,NaN
3,CXR\n No previous film to be compared.\n RLL...,"['<s> cxr <\\s>', '<s> no previous film to be ...",1,0,NaN,NaN
4,CXR PA upright\n The chest shows mild reticul...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",1,0,NaN,NaN
...,...,...,...,...,...,...
34342,CXR PA\n\nNo change of nodular shadow at left ...,"['<s> cxr pa <\\s>', '<s> no change of nodular...",0,1,NaN,NaN
34343,CXR PA upright\n\n DIffuse reticulonodular in...,"['<s> cxr pa upright <\\s>', '<s> diffuse reti...",1,0,NaN,NaN
34344,CXR PA \n\nDiffuse reticulonodular infiltratio...,"['<s> cxr pa <\\s>', '<s> diffuse reticulonodu...",1,0,NaN,NaN
34345,Chest PA upright \n\nknown CA breast\n\nSurgic...,"['<s> chest pa upright <\\s>', '<s> known ca b...",1,0,NaN,NaN


In [241]:
# create report list
lesion_report = createReportList(lesion_df)
lesion_report[0]

['<s> cxr pa upright <\\s>',
 '<s> reticulonodular infiltration at both upper lobes <\\s>',
 '<s> the heart and great vessels appear normal <\\s>',
 '<s> no pleural effusion <\\s>',
 '<s> bony thorax is intact <\\s>',
 '<s> imp tb is considered <\\s>',
 '<s> clinical and previous film correlation are <\\s>',
 '<s> suggested <\\s>']

Find N-grams Pattern

In [242]:
lesion_bigram = createPatternNgrams(lesion_report, 2)
list(lesion_bigram.items())[:20]

[(('<s>', 'no'), 21247),
 (('infiltration', 'at'), 11957),
 (('bony', 'thorax'), 11760),
 (('pleural', 'effusion'), 11220),
 (('<s>', 'the'), 10457),
 (('reticulonodular', 'infiltration'), 9658),
 (('intact', '<\\s>'), 9205),
 (('thorax', 'is'), 8827),
 (('is', 'intact'), 8327),
 (('<s>', 'bony'), 7860),
 (('effusion', '<\\s>'), 7552),
 (('is', 'seen'), 7380),
 (('seen', '<\\s>'), 7257),
 (('noted', '<\\s>'), 6920),
 (('clear', '<\\s>'), 6841),
 (('no', 'pleural'), 6814),
 (('both', 'costophrenic'), 6519),
 (('<s>', 'both'), 6236),
 (('<s>', 'normal'), 6175),
 (('is', 'noted'), 6155)]

In [243]:
lesion_trigram = createPatternNgrams(lesion_report, 3)
list(lesion_trigram.items())[:20]

[(('bony', 'thorax', 'is'), 8810),
 (('is', 'intact', '<\\s>'), 8325),
 (('thorax', 'is', 'intact'), 7923),
 (('pleural', 'effusion', '<\\s>'), 7224),
 (('<s>', 'bony', 'thorax'), 7094),
 (('no', 'pleural', 'effusion'), 6794),
 (('reticulonodular', 'infiltration', 'at'), 6771),
 (('<s>', 'no', 'pleural'), 6424),
 (('is', 'seen', '<\\s>'), 6327),
 (('are', 'clear', '<\\s>'), 5662),
 (('<s>', 'both', 'costophrenic'), 5637),
 (('<s>', 'no', 'cardiomegaly'), 5488),
 (('is', 'noted', '<\\s>'), 5406),
 (('costophrenic', 'angles', 'are'), 4424),
 (('<s>', 'there', 'is'), 4194),
 (('both', 'costophrenic', 'angles'), 3987),
 (('angles', 'are', 'clear'), 3669),
 (('<s>', 'reticulonodular', 'infiltration'), 3315),
 (('<s>', 'the', 'heart'), 3311),
 (('no', 'cardiomegaly', '<\\s>'), 3290)]

In [244]:
lesion_fourgram = createPatternNgrams(lesion_report, 4)
list(lesion_fourgram.items())[:20]

[(('thorax', 'is', 'intact', '<\\s>'), 7921),
 (('bony', 'thorax', 'is', 'intact'), 7916),
 (('<s>', 'bony', 'thorax', 'is'), 6591),
 (('<s>', 'no', 'pleural', 'effusion'), 6405),
 (('no', 'pleural', 'effusion', '<\\s>'), 4958),
 (('both', 'costophrenic', 'angles', 'are'), 3819),
 (('costophrenic', 'angles', 'are', 'clear'), 3669),
 (('angles', 'are', 'clear', '<\\s>'), 3662),
 (('<s>', 'both', 'costophrenic', 'angles'), 3216),
 (('<s>', 'normal', 'cardiothoracic', 'ratio'), 2730),
 (('<s>', 'no', 'cardiomegaly', '<\\s>'), 2695),
 (('are', 'not', 'remarkable', '<\\s>'), 2659),
 (('normal', 'cardiothoracic', 'ratio', '<\\s>'), 2539),
 (('<s>', 'reticulonodular', 'infiltration', 'at'), 2436),
 (('<s>', 'both', 'costophrenic', 'sulci'), 2392),
 (('<s>', 'no', 'cardiomegaly', 'is'), 2353),
 (('is', 'not', 'enlarged', '<\\s>'), 2333),
 (('reticulonodular', 'infiltration', 'at', 'both'), 2104),
 (('<s>', 'the', 'bony', 'thorax'), 2063),
 (('the', 'bony', 'thorax', 'is'), 2040)]

In [245]:
lesion_bigramk = createPatternNgrams(lesion_report, 2, lesion_keyword)
list(lesion_bigramk.items())[:20]

[(('<s>', 'history'), 2),
 (('history', 'a'), 2),
 (('old', 'man'), 2),
 (('mass', 'was'), 2),
 (('was', 'sent'), 2),
 (('sent', 'for'), 2),
 (('evaluation', '<\\s>'), 2),
 (('a', '63'), 1),
 (('63', 'year'), 1),
 (('year', 'old'), 1),
 (('man', 'with'), 1),
 (('with', 'large'), 1),
 (('large', 'retroperitoneal'), 1),
 (('retroperitoneal', 'mass'), 1),
 (('for', 'evaluation'), 1),
 (('<s>', 'widening'), 1),
 (('widening', 'of'), 1),
 (('of', 'mediastinum'), 1),
 (('mediastinum', 'silhouette'), 1),
 (('silhouette', 'with'), 1)]

In [246]:
lesion_trigramk = createPatternNgrams(lesion_report, 3, lesion_keyword)
list(lesion_trigramk.items())[:20]

[(('<s>', 'history', 'a'), 2),
 (('mass', 'was', 'sent'), 2),
 (('was', 'sent', 'for'), 2),
 (('history', 'a', '63'), 1),
 (('a', '63', 'year'), 1),
 (('63', 'year', 'old'), 1),
 (('year', 'old', 'man'), 1),
 (('old', 'man', 'with'), 1),
 (('man', 'with', 'large'), 1),
 (('with', 'large', 'retroperitoneal'), 1),
 (('large', 'retroperitoneal', 'mass'), 1),
 (('retroperitoneal', 'mass', 'was'), 1),
 (('sent', 'for', 'evaluation'), 1),
 (('for', 'evaluation', '<\\s>'), 1),
 (('<s>', 'widening', 'of'), 1),
 (('widening', 'of', 'mediastinum'), 1),
 (('of', 'mediastinum', 'silhouette'), 1),
 (('mediastinum', 'silhouette', 'with'), 1),
 (('silhouette', 'with', 'cardiac'), 1),
 (('with', 'cardiac', 'border'), 1)]

In [247]:
lesion_fourgramk = createPatternNgrams(lesion_report, 4, lesion_keyword)
list(lesion_fourgramk.items())[:20]

[(('mass', 'was', 'sent', 'for'), 2),
 (('<s>', 'history', 'a', '63'), 1),
 (('history', 'a', '63', 'year'), 1),
 (('a', '63', 'year', 'old'), 1),
 (('63', 'year', 'old', 'man'), 1),
 (('year', 'old', 'man', 'with'), 1),
 (('old', 'man', 'with', 'large'), 1),
 (('man', 'with', 'large', 'retroperitoneal'), 1),
 (('with', 'large', 'retroperitoneal', 'mass'), 1),
 (('large', 'retroperitoneal', 'mass', 'was'), 1),
 (('retroperitoneal', 'mass', 'was', 'sent'), 1),
 (('was', 'sent', 'for', 'evaluation'), 1),
 (('sent', 'for', 'evaluation', '<\\s>'), 1),
 (('<s>', 'widening', 'of', 'mediastinum'), 1),
 (('widening', 'of', 'mediastinum', 'silhouette'), 1),
 (('of', 'mediastinum', 'silhouette', 'with'), 1),
 (('mediastinum', 'silhouette', 'with', 'cardiac'), 1),
 (('silhouette', 'with', 'cardiac', 'border'), 1),
 (('with', 'cardiac', 'border', 'mediastinal'), 1),
 (('cardiac', 'border', 'mediastinal', 'mass'), 1)]

In [248]:
lesion_fivegramk = createPatternNgrams(lesion_report, 5, lesion_keyword)
list(lesion_fivegramk.items())[:20]

[(('<s>', 'history', 'a', '63', 'year'), 1),
 (('history', 'a', '63', 'year', 'old'), 1),
 (('a', '63', 'year', 'old', 'man'), 1),
 (('63', 'year', 'old', 'man', 'with'), 1),
 (('year', 'old', 'man', 'with', 'large'), 1),
 (('old', 'man', 'with', 'large', 'retroperitoneal'), 1),
 (('man', 'with', 'large', 'retroperitoneal', 'mass'), 1),
 (('with', 'large', 'retroperitoneal', 'mass', 'was'), 1),
 (('large', 'retroperitoneal', 'mass', 'was', 'sent'), 1),
 (('retroperitoneal', 'mass', 'was', 'sent', 'for'), 1),
 (('mass', 'was', 'sent', 'for', 'evaluation'), 1),
 (('was', 'sent', 'for', 'evaluation', '<\\s>'), 1),
 (('<s>', 'widening', 'of', 'mediastinum', 'silhouette'), 1),
 (('widening', 'of', 'mediastinum', 'silhouette', 'with'), 1),
 (('of', 'mediastinum', 'silhouette', 'with', 'cardiac'), 1),
 (('mediastinum', 'silhouette', 'with', 'cardiac', 'border'), 1),
 (('silhouette', 'with', 'cardiac', 'border', 'mediastinal'), 1),
 (('with', 'cardiac', 'border', 'mediastinal', 'mass'), 1),
 (

In [249]:
lesion_sixgramk = createPatternNgrams(lesion_report, 6, lesion_keyword)
list(lesion_sixgramk.items())[:20]

[(('<s>', 'history', 'a', '63', 'year', 'old'), 1),
 (('history', 'a', '63', 'year', 'old', 'man'), 1),
 (('a', '63', 'year', 'old', 'man', 'with'), 1),
 (('63', 'year', 'old', 'man', 'with', 'large'), 1),
 (('year', 'old', 'man', 'with', 'large', 'retroperitoneal'), 1),
 (('old', 'man', 'with', 'large', 'retroperitoneal', 'mass'), 1),
 (('man', 'with', 'large', 'retroperitoneal', 'mass', 'was'), 1),
 (('with', 'large', 'retroperitoneal', 'mass', 'was', 'sent'), 1),
 (('large', 'retroperitoneal', 'mass', 'was', 'sent', 'for'), 1),
 (('retroperitoneal', 'mass', 'was', 'sent', 'for', 'evaluation'), 1),
 (('mass', 'was', 'sent', 'for', 'evaluation', '<\\s>'), 1),
 (('<s>', 'widening', 'of', 'mediastinum', 'silhouette', 'with'), 1),
 (('widening', 'of', 'mediastinum', 'silhouette', 'with', 'cardiac'), 1),
 (('of', 'mediastinum', 'silhouette', 'with', 'cardiac', 'border'), 1),
 (('mediastinum', 'silhouette', 'with', 'cardiac', 'border', 'mediastinal'),
  1),
 (('silhouette', 'with', 'cardia

## Difference between Result 2 and 3

In [250]:
findDiffData('Lung Lesion', lesion_data_old, lesion_data)

nan 1.0 ['<s> cxr pa upright <\\s>', '<s> the chest shows multiple nodular infiltrations at both lungs metastasis should be considered <\\s>', '<s> the nodules are not seen on previous study <\\s>', '<s> the heart is not enlarged <\\s>', '<s> both costophrenic angles are clear <\\s>', '<s> the bony thorax is intact <\\s>']
nan 1.0 ['<s> pa cxr <\\s>', '<s> clinical history pre op on thyroid mass <\\s>', '<s> no definite pulmonary infiltration <\\s>', '<s> the heart and great vessels appear normal <\\s>', '<s> bilateral costophrenic angles are unremarkable <\\s>', '<s> bony thorax is intact <\\s>', '<s> imp normal chest <\\s>']
nan 1.0 ['<s> cxr <\\s>', '<s> clinical history pre op tonsillar mass <\\s>', '<s> no definite pulmonary infiltration <\\s>', '<s> the heart and great vessels appear normal <\\s>', '<s> bilateral costophrenic angles are unremarkable <\\s>', '<s> bony thorax is intact <\\s>', '<s> imp normal chest <\\s>']
nan 1.0 ['<s> cxr pa upright <\\s>', '<s> history thyroid n

In [251]:
findDiffData2('Lung Lesion', lesion_data_old, lesion_data)

nan 1.0 ['<s> cxr pa upright <\\s>', '<s> the chest shows multiple nodular infiltrations at both lungs metastasis should be considered <\\s>', '<s> the nodules are not seen on previous study <\\s>', '<s> the heart is not enlarged <\\s>', '<s> both costophrenic angles are clear <\\s>', '<s> the bony thorax is intact <\\s>']
         ['<s> cxr pa upright <\\s>', '<s> the chest shows multiple nodular infiltrations at both lungs metastasis should be considered <\\s>', '<s> the nodules are not seen on previous study <\\s>', '<s> the heart is not enlarged <\\s>', '<s> both costophrenic angles are clear <\\s>', '<s> the bony thorax is intact <\\s>']
nan 1.0 ['<s> pa cxr <\\s>', '<s> clinical history pre op on thyroidal mass <\\s>', '<s> no definite pulmonary infiltration <\\s>', '<s> the heart and great vessels appear normal <\\s>', '<s> bilateral costophrenic angles are unremarkable <\\s>', '<s> bony thorax is intact <\\s>', '<s> imp normal chest <\\s>']
         ['<s> pa cxr <\\s>', '<s> cl

# Inspectra Lung Opacity v1

In [252]:
# define keyword
infil_keyword = r'(infil|densi|opaci|'
conso_keyword = 'consolidat|'
opaci_keyword = 'opaci|translu|air|mark|pattern|lung|reticul|scar|thicken|densi|patch)'
v1_keyword = infil_keyword + conso_keyword + opaci_keyword

## Ground truth 0

Prepare Data

In [253]:
# choose reports
v1_df_0 = selectData(v1_data, 'Inspectra Lung Opacity v1', 0.0)
v1_df_0

,Reports,Report List,Inspectra Lung Opacity v1 BERT Labeler,Inspectra Lung Opacity v1 Inspectra Labeler,Inspectra Lung Opacity v1 Ground Truth,Inspectra Lung Opacity v1 Key Detect
13,Chest film\n\nNo definite pulmonary infiltrati...,"['<s> chest film <\\s>', '<s> no definite pulm...",0,1,0.0,"<re.Match object; span=(4, 30), match='no defi..."
43,CXR (PA upright)\n\nNo definite pulmonary inf...,"['<s> cxr pa upright <\\s>', '<s> no definite ...",0,1,0.0,"<re.Match object; span=(4, 30), match='no defi..."
61,CXR PA upright\n The chest shows no definite ...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,0.0,"<re.Match object; span=(20, 46), match='no def..."
97,CXR \n\nHistory of chronic cough\n\nNo definit...,"['<s> cxr <\\s>', '<s> history of chronic coug...",0,1,0.0,"<re.Match object; span=(4, 30), match='no defi..."
101,CXR\n No pulmonary infiltration or nodule.\n ...,"['<s> cxr <\\s>', '<s> no pulmonary infiltrati...",0,1,0.0,"<re.Match object; span=(4, 21), match='no pulm..."
...,...,...,...,...,...,...
76631,CXR PA upright\n\n There is no definite pulmo...,"['<s> cxr pa upright <\\s>', '<s> there is no ...",0,1,0.0,"<re.Match object; span=(13, 39), match='no def..."
76645,Chest PA upright\n\nFindings:\nNo definite pul...,"['<s> chest pa upright <\\s>', '<s> findings <...",0,1,0.0,"<re.Match object; span=(4, 30), match='no defi..."
76649,Chest\nNo new active infiltration\nUnchanged i...,"['<s> chest <\\s>', '<s> no new active infiltr...",0,1,0.0,"<re.Match object; span=(4, 22), match='no new ..."
76651,CXR PA UPRIGHT\n\nFINDINGS : No detectable pu...,"['<s> cxr pa upright <\\s>', '<s> findings no ...",0,1,0.0,"<re.Match object; span=(13, 72), match='no det..."


In [254]:
# create report list
v1_report_0 = createReportList(v1_df_0)
v1_report_0[0]

['<s> chest film <\\s>',
 '<s> no definite pulmonary infiltration <\\s>',
 '<s> radiodensity line in rul likely skin fold <\\s>',
 '<s> cardiomegaly with dilated aorta <\\s>',
 '<s> right paratracheal density could be vascular shadow <\\s>',
 '<s> both costophrenic angles are clear <\\s>',
 '<s> old fracture right 8th rib <\\s>']

Find N-grams Pattern

In [255]:
v1_bigram_0 = createPatternNgrams(v1_report_0, 2, v1_keyword)
list(v1_bigram_0.items())[:20]

[(('pulmonary', 'infiltration'), 30687),
 (('infiltration', 'or'), 15366),
 (('active', 'pulmonary'), 15288),
 (('or', 'nodule'), 15122),
 (('infiltration', '<\\s>'), 14568),
 (('no', 'definite'), 14510),
 (('definite', 'pulmonary'), 13715),
 (('there', 'is'), 13416),
 (('no', 'active'), 13057),
 (('is', 'no'), 13055),
 (('<s>', 'there'), 13045),
 (('<s>', 'imp'), 12808),
 (('nodule', '<\\s>'), 12802),
 (('imp', 'no'), 12790),
 (('noted', '<\\s>'), 4251),
 (('is', 'noted'), 3864),
 (('<s>', 'no'), 3655),
 (('<s>', 'findings'), 3095),
 (('infiltration', 'at'), 2975),
 (('no', 'detectable'), 2772)]

In [256]:
v1_trigram_0 = createPatternNgrams(v1_report_0, 3, v1_keyword)
list(v1_trigram_0.items())[:20]

[(('pulmonary', 'infiltration', 'or'), 15324),
 (('active', 'pulmonary', 'infiltration'), 15285),
 (('infiltration', 'or', 'nodule'), 15116),
 (('pulmonary', 'infiltration', '<\\s>'), 14378),
 (('no', 'definite', 'pulmonary'), 13712),
 (('definite', 'pulmonary', 'infiltration'), 13536),
 (('there', 'is', 'no'), 13048),
 (('no', 'active', 'pulmonary'), 13015),
 (('<s>', 'there', 'is'), 13011),
 (('is', 'no', 'definite'), 12795),
 (('<s>', 'imp', 'no'), 12790),
 (('or', 'nodule', '<\\s>'), 12743),
 (('imp', 'no', 'active'), 12420),
 (('is', 'noted', '<\\s>'), 3649),
 (('<s>', 'findings', 'no'), 2699),
 (('findings', 'no', 'detectable'), 2676),
 (('or', 'nodule', 'is'), 2127),
 (('nodule', 'is', 'noted'), 2077),
 (('fibrotic', 'infiltration', 'at'), 1803),
 (('is', 'seen', '<\\s>'), 1746)]

## Ground truth 1

Prepare Data

In [257]:
# choose reports
v1_df_1 = selectData(v1_data, 'Inspectra Lung Opacity v1', 1.0)
v1_df_1

,Reports,Report List,Inspectra Lung Opacity v1 BERT Labeler,Inspectra Lung Opacity v1 Inspectra Labeler,Inspectra Lung Opacity v1 Ground Truth,Inspectra Lung Opacity v1 Key Detect
0,CXR (PA upright)\n\nPatchy infiltration at RL...,"['<s> cxr pa upright <\\s>', '<s> patchy infil...",0,1,1.0,"<re.Match object; span=(0, 15), match='<s> pat..."
1,CXR (PA upright)\n\nReticulonodular infiltrati...,"['<s> cxr pa upright <\\s>', '<s> reticulonodu...",0,1,1.0,"<re.Match object; span=(0, 24), match='<s> ret..."
2,Chest:-\nPa chest study reveals fibronodular i...,"['<s> chest <\\s>', '<s> pa chest study reveal...",0,1,1.0,"<re.Match object; span=(0, 44), match='<s> pa ..."
3,CHEST :\nP.A. upright view .\nFibronodular inf...,"['<s> chest <\\s>', '<s> p.a <\\s>', '<s> upri...",0,1,1.0,"<re.Match object; span=(0, 21), match='<s> fib..."
4,CHEST :\nP.A. upright view .\nFibronodular inf...,"['<s> chest <\\s>', '<s> p.a <\\s>', '<s> upri...",0,1,1.0,"<re.Match object; span=(0, 21), match='<s> fib..."
...,...,...,...,...,...,...
76653,Chest PA upright \n\nknown CA breast\n\nSurgic...,"['<s> chest pa upright <\\s>', '<s> known ca b...",0,1,1.0,"<re.Match object; span=(0, 66), match='<s> fib..."
76655,CXR PA\n\nThere is interstitial infiltation at...,"['<s> cxr pa <\\s>', '<s> there is interstitia...",0,1,1.0,"<re.Match object; span=(0, 30), match='<s> the..."
76657,CXR(PA)\n \n Reticulocalcified infiltration...,"['<s> cxr pa <\\s>', '<s> reticulocalcified in...",0,1,1.0,"<re.Match object; span=(0, 26), match='<s> ret..."
76658,CXR(PA)\n\n IMP: Retioculonodular infiltratio...,"['<s> cxr pa <\\s>', '<s> imp reticulonodular ...",0,1,1.0,"<re.Match object; span=(0, 28), match='<s> imp..."


In [258]:
# create report list
v1_report_1 = createReportList(v1_df_1)
v1_report_1[0]

['<s> cxr pa upright <\\s>',
 '<s> patchy infiltration at rll lul and lll <\\s>',
 '<s> mild cardiomegaly <\\s>',
 '<s> no perihilar or mediastinal mass <\\s>',
 '<s> no pleural effusion <\\s>',
 '<s> bony thorax is intact <\\s>']

Find N-grams Pattern

In [259]:
v1_bigram_1 = createPatternNgrams(v1_report_1, 2, v1_keyword)
list(v1_bigram_1.items())[:20]

[(('infiltration', 'at'), 12457),
 (('reticulonodular', 'infiltration'), 6961),
 (('at', 'both'), 6783),
 (('at', 'right'), 5225),
 (('at', 'rul'), 4873),
 (('<s>', 'reticulonodular'), 4534),
 (('both', 'lungs'), 4283),
 (('noted', '<\\s>'), 4241),
 (('is', 'seen'), 4086),
 (('is', 'noted'), 3700),
 (('pleural', 'thickening'), 3594),
 (('unremarkable', '<\\s>'), 3533),
 (('seen', '<\\s>'), 3518),
 (('<s>', 'no'), 3432),
 (('at', 'left'), 3385),
 (('opacity', 'at'), 3225),
 (('pulmonary', 'infiltration'), 3123),
 (('<s>', 'there'), 3075),
 (('interstitial', 'infiltration'), 3058),
 (('there', 'is'), 3006)]

In [260]:
v1_trigram_1 = createPatternNgrams(v1_report_1, 3, v1_keyword)
list(v1_trigram_1.items())[:20]

[(('reticulonodular', 'infiltration', 'at'), 4683),
 (('infiltration', 'at', 'both'), 3743),
 (('<s>', 'reticulonodular', 'infiltration'), 3393),
 (('infiltration', 'at', 'rul'), 2772),
 (('<s>', 'there', 'is'), 2598),
 (('is', 'noted', '<\\s>'), 2510),
 (('is', 'seen', '<\\s>'), 2362),
 (('not', 'remarkable', '<\\s>'), 2244),
 (('are', 'not', 'remarkable'), 2216),
 (('at', 'both', 'upper'), 2155),
 (('at', 'both', 'lungs'), 2138),
 (('at', 'rul', '<\\s>'), 1979),
 (('interstitial', 'infiltration', 'at'), 1962),
 (('both', 'upper', 'lobes'), 1747),
 (('costophrenic', 'sulci', 'are'), 1714),
 (('sulci', 'are', 'not'), 1700),
 (('infiltration', 'at', 'right'), 1688),
 (('both', 'lungs', '<\\s>'), 1633),
 (('apical', 'pleural', 'thickening'), 1593),
 (('nodular', 'opacity', 'at'), 1555)]

## Ground truth -1

Prepare Data

In [261]:
# choose reports
v1_df_u = selectData(v1_data, 'Inspectra Lung Opacity v1', -1.0)
v1_df_u

,Reports,Report List,Inspectra Lung Opacity v1 BERT Labeler,Inspectra Lung Opacity v1 Inspectra Labeler,Inspectra Lung Opacity v1 Ground Truth,Inspectra Lung Opacity v1 Key Detect
6,CXR PA upright\n The chest shows mild reticul...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,-1.0,"<re.Match object; span=(20, 32), match='mild r..."
11,CXR\n Multiple nodules are noted at left lung...,"['<s> cxr <\\s>', '<s> multiple nodules are no...",0,1,-1.0,"<re.Match object; span=(52, 66), match='sugges..."
12,Chest;\n\n\n No significant change of reticul...,"['<s> chest <\\s>', '<s> no significant change...",0,1,-1.0,"<re.Match object; span=(4, 80), match='no sign..."
14,Chest film\n\nUnchanged fibrocalcific infiltra...,"['<s> chest film <\\s>', '<s> unchanged fibroc...",0,1,-1.0,"<re.Match object; span=(4, 82), match='unchang..."
31,CXR\n\nHistory of rule out TB\n\nAs compared t...,"['<s> cxr <\\s>', '<s> history of rule out tb ...",0,1,-1.0,"<re.Match object; span=(61, 194), match='no si..."
...,...,...,...,...,...,...
76641,Chest CXR\n\nNo change of a tiny nodular opaci...,"['<s> chest cxr <\\s>', '<s> no change of a ti...",0,1,-1.0,"<re.Match object; span=(4, 60), match='no chan..."
76644,CXR PA UPRIGHT\nCompared with 27/09/2017\n\n- ...,"['<s> cxr pa upright <\\s>', '<s> compared wit...",0,1,-1.0,"<re.Match object; span=(4, 49), match='no sign..."
76648,CXR PA\n\nNo change of nodular shadow at left ...,"['<s> cxr pa <\\s>', '<s> no change of nodular...",0,1,-1.0,"<re.Match object; span=(4, 80), match='no chan..."
76654,"Chest:\nNo change of emphysematous lung, RUL a...","['<s> chest <\\s>', '<s> no change of emphysem...",0,1,-1.0,"<re.Match object; span=(4, 52), match='no chan..."


In [262]:
# create report list
v1_report_u = createReportList(v1_df_u)
v1_report_u[0]

['<s> cxr pa upright <\\s>',
 '<s> the chest shows mild reticulonodular infiltration at right apical lung <\\s>',
 '<s> the left lung is clear <\\s>',
 '<s> the heart is not enlarged <\\s>',
 '<s> both costophrenic angles are clear <\\s>',
 '<s> the bony thorax is intact <\\s>']

Find N-grams Pattern

In [263]:
v1_bigram_u = createPatternNgrams(v1_report_u, 2, v1_keyword)
list(v1_bigram_u.items())[:20]

[(('change', 'of'), 20293),
 (('<s>', 'no'), 15555),
 (('infiltration', 'at'), 13196),
 (('no', 'change'), 10915),
 (('no', 'significant'), 10316),
 (('significant', 'change'), 9698),
 (('at', 'right'), 8003),
 (('<s>', 'the'), 6835),
 (('at', 'both'), 6775),
 (('reticulonodular', 'infiltration'), 5133),
 (('lung', '<\\s>'), 5082),
 (('at', 'left'), 5012),
 (('unremarkable', '<\\s>'), 4408),
 (('at', 'rul'), 4358),
 (('pleural', 'thickening'), 4331),
 (('<s>', 'unchanged'), 4263),
 (('interstitial', 'infiltration'), 4054),
 (('the', 'chest'), 3977),
 (('chest', 'shows'), 3968),
 (('noted', '<\\s>'), 3667)]

In [264]:
v1_trigram_u = createPatternNgrams(v1_report_u, 3, v1_keyword)
list(v1_trigram_u.items())[:20]

[(('no', 'change', 'of'), 10510),
 (('no', 'significant', 'change'), 9689),
 (('significant', 'change', 'of'), 9221),
 (('<s>', 'no', 'significant'), 7512),
 (('<s>', 'no', 'change'), 5203),
 (('<s>', 'the', 'chest'), 3976),
 (('the', 'chest', 'shows'), 3968),
 (('reticulonodular', 'infiltration', 'at'), 3736),
 (('infiltration', 'at', 'both'), 3435),
 (('chest', 'shows', 'no'), 3287),
 (('shows', 'no', 'change'), 3076),
 (('<s>', 'there', 'is'), 2963),
 (('infiltration', 'at', 'right'), 2810),
 (('interstitial', 'infiltration', 'at'), 2571),
 (('is', 'noted', '<\\s>'), 2468),
 (('infiltration', 'at', 'rul'), 2416),
 (('change', 'of', 'interstitial'), 2353),
 (('of', 'reticulonodular', 'infiltration'), 2134),
 (('there', 'is', 'no'), 2065),
 (('is', 'unremarkable', '<\\s>'), 1960)]

## Unlabel

Prepare Data

In [265]:
# choose reports
v1_df = selectData(v1_data, 'Inspectra Lung Opacity v1', None)
v1_df

,Reports,Report List,Inspectra Lung Opacity v1 BERT Labeler,Inspectra Lung Opacity v1 Inspectra Labeler,Inspectra Lung Opacity v1 Ground Truth,Inspectra Lung Opacity v1 Key Detect
534,CXR PA upright\n\nFibronfiltration at LUL\nFib...,"['<s> cxr pa upright <\\s>', '<s> fibronfiltra...",0,1,NaN,NaN
674,CHEST PA\n\nSoft tissue mass at the LUL region...,"['<s> chest pa <\\s>', '<s> soft tissue mass a...",1,0,NaN,NaN
1146,CHEST PA\n\n There is round lucency superimpos...,"['<s> chest pa <\\s>', '<s> there is round lic...",1,0,NaN,NaN
3480,"CHEST.;\n\nCardiomegaly,minimal perihilar hazi...","['<s> chest <\\s>', '<s> cardiomegaly minimal ...",1,0,NaN,NaN
4872,CXR\n\nMild cardiomegaly with calcified aortic...,"['<s> cxr <\\s>', '<s> mild cardiomegaly with ...",1,0,NaN,NaN
...,...,...,...,...,...,...
74919,CXR PA upright\n\n Bronchiectasis/atelectasis...,"['<s> cxr pa upright <\\s>', '<s> bronchiectas...",1,0,NaN,NaN
74937,CXR PA upright\n\n Minimal bronchiectasis at ...,"['<s> cxr pa upright <\\s>', '<s> minimal bron...",1,0,NaN,NaN
75409,CHEST XRAY\n\n- No significant change of bony ...,"['<s> chest ray <\\s>', '<s> no significant ch...",1,0,NaN,NaN
76068,"Chest:PA,upright\nCardiomegaly is noted.\nPulm...","['<s> chest pa upright <\\s>', '<s> cardiomega...",1,0,NaN,NaN


In [266]:
# create report list
v1_report = createReportList(v1_df)
v1_report[0]

['<s> cxr pa upright <\\s>',
 '<s> fibronfiltration at lul <\\s>',
 '<s> fibrosis rll <\\s>',
 '<s> normal cardiothoracic ratio <\\s>',
 '<s> bony thorax is intact <\\s>',
 '<s> elevation of right hemidiaphragm <\\s>']

In [267]:
v1_report[1:5]

[['<s> chest pa <\\s>',
  '<s> soft tissue mass at the lul region with destruction the left 3rd rib <\\s>',
  '<s> old fx of the right posterior 6 7 th rib <\\s>',
  '<s> navigating lesion at the periphery of the rul <\\s>',
  '<s> left effusion <\\s>'],
 ['<s> chest pa <\\s>',
  '<s> there is round licence superimpose with the cardiac shadow could be hiatal hernia or morgan hernia <\\s>'],
 ['<s> chest <\\s>',
  '<s> cardiomegaly minimal perihilar harnesses bilaterally <\\s>',
  '<s> suggestion of congestive heart <\\s>'],
 ['<s> cxr <\\s>',
  '<s> mild cardiomegaly with calcified aortic knob <\\s>',
  '<s> unchanged lt. retrocardaic haziness with bronchogram is noted <\\s>',
  '<s> intact bony thorax <\\s>']]

Find N-grams Pattern

In [268]:
v1_bigram = createPatternNgrams(v1_report, 2)
list(v1_bigram.items())[:20]

[(('<s>', 'no'), 34),
 (('<s>', 'the'), 30),
 (('<s>', 'cardiomegaly'), 27),
 (('bony', 'thorax'), 26),
 (('intact', '<\\s>'), 26),
 (('pleural', 'effusion'), 25),
 (('costophrenic', 'angle'), 23),
 (('<s>', 'normal'), 22),
 (('cardiomegaly', '<\\s>'), 22),
 (('blunting', 'of'), 22),
 (('thorax', 'is'), 21),
 (('is', 'intact'), 21),
 (('effusion', '<\\s>'), 19),
 (('<s>', 'unchanged'), 19),
 (('<s>', 'mild'), 17),
 (('both', 'costophrenic'), 17),
 (('noted', '<\\s>'), 16),
 (('<s>', 'there'), 15),
 (('mild', 'cardiomegaly'), 15),
 (('costophrenic', 'angles'), 15)]

In [269]:
v1_trigram = createPatternNgrams(v1_report, 3)
list(v1_trigram.items())[:20]

[(('bony', 'thorax', 'is'), 21),
 (('thorax', 'is', 'intact'), 21),
 (('is', 'intact', '<\\s>'), 21),
 (('pleural', 'effusion', '<\\s>'), 17),
 (('both', 'costophrenic', 'angles'), 15),
 (('<s>', 'mild', 'cardiomegaly'), 13),
 (('costophrenic', 'angle', '<\\s>'), 13),
 (('<s>', 'there', 'is'), 12),
 (('is', 'noted', '<\\s>'), 12),
 (('left', 'costophrenic', 'angle'), 12),
 (('are', 'clear', '<\\s>'), 11),
 (('<s>', 'cardiomegaly', '<\\s>'), 11),
 (('<s>', 'unchanged', 'of'), 10),
 (('is', 'seen', '<\\s>'), 10),
 (('<s>', 'both', 'costophrenic'), 10),
 (('costophrenic', 'angles', 'are'), 10),
 (('right', 'costophrenic', 'angle'), 9),
 (('<s>', 'cardiomegaly', 'is'), 9),
 (('<s>', 'no', 'pleural'), 9),
 (('no', 'pleural', 'effusion'), 9)]

In [270]:
v1_fourgram = createPatternNgrams(v1_report, 4)
list(v1_fourgram.items())[:20]

[(('bony', 'thorax', 'is', 'intact'), 21),
 (('thorax', 'is', 'intact', '<\\s>'), 21),
 (('both', 'costophrenic', 'angles', 'are'), 10),
 (('left', 'costophrenic', 'angle', '<\\s>'), 9),
 (('<s>', 'no', 'pleural', 'effusion'), 9),
 (('<s>', 'both', 'costophrenic', 'angles'), 9),
 (('costophrenic', 'angles', 'are', 'clear'), 9),
 (('angles', 'are', 'clear', '<\\s>'), 9),
 (('<s>', 'the', 'bony', 'thorax'), 9),
 (('<s>', 'bony', 'thorax', 'is'), 8),
 (('blunting', 'of', 'left', 'costophrenic'), 8),
 (('of', 'left', 'costophrenic', 'angle'), 8),
 (('no', 'pleural', 'effusion', '<\\s>'), 8),
 (('the', 'bony', 'thorax', 'is'), 8),
 (('of', 'right', 'costophrenic', 'angle'), 7),
 (('<s>', 'minimal', 'blunting', 'of'), 7),
 (('cardiomegaly', 'is', 'noted', '<\\s>'), 7),
 (('<s>', 'normal', 'cardiothoracic', 'ratio'), 6),
 (('normal', 'cardiothoracic', 'ratio', '<\\s>'), 6),
 (('<s>', 'no', 'cardiomegaly', '<\\s>'), 6)]

In [271]:
v1_bigramk = createPatternNgrams(v1_report, 2, v1_keyword)
list(v1_bigramk.items())[:20]

[]

In [272]:
v1_trigramk = createPatternNgrams(v1_report, 3, v1_keyword)
list(v1_trigramk.items())[:20]

[]

In [273]:
v1_fourgramk = createPatternNgrams(v1_report, 4, v1_keyword)
list(v1_fourgramk.items())[:20]

[]

## Difference between Result 2 and 3

In [274]:
findDiffData('Inspectra Lung Opacity v1', v1_data_old, v1_data)

1.0 -1.0 ['<s> cxr pa upright <\\s>', '<s> the chest shows mild reticulonodular infiltration at both apical lung mild nodular infiltration at left upper lung and right lower lung compatible with history of tb <\\s>', '<s> the heart is not enlarged <\\s>', '<s> both costophrenic angles are clear <\\s>', '<s> the bony thorax is intact <\\s>']
0.0 -1.0 ['<s> cxr pa upright <\\s>', '<s> by compared with previous chest on 10 05 2005 <\\s>', '<s> the chest shows no change of opacity at right lower lung <\\s>', '<s> the remaining lungs and left costophrenic angles are clear <\\s>', '<s> the heart is mild enlarged <\\s>', '<s> the bony thorax is intact <\\s>']
1.0 -1.0 ['<s> cxr pa upright <\\s>', '<s> by compared with previous chest on 29 08 2005 <\\s>', '<s> the chest shows decreased in density of nodule at right upper lung size about 1.5 1.8 cm <\\s>', '<s> there is also no change of round small nodule at left upper lung about 0.7 cm <\\s>', '<s> the prominent hila are no change <\\s>']
0.0

In [275]:
findDiffData2('Inspectra Lung Opacity v1', v1_data_old, v1_data)

1.0 -1.0 ['<s> cxr pa upright <\\s>', '<s> the chest shows mild reitculonodular infiltration at both apical lung mild nodular infiltration at left upper lung and right lower lung compatible with history of tb <\\s>', '<s> the heart is not enlarged <\\s>', '<s> both costophrenic angles are clear <\\s>', '<s> the bony thorax is intact <\\s>']
         ['<s> cxr pa upright <\\s>', '<s> the chest shows mild reticulonodular infiltration at both apical lung mild nodular infiltration at left upper lung and right lower lung compatible with history of tb <\\s>', '<s> the heart is not enlarged <\\s>', '<s> both costophrenic angles are clear <\\s>', '<s> the bony thorax is intact <\\s>']
0.0 -1.0 ['<s> cxr pa upright <\\s>', '<s> by compared with previous chest on 10 05 2005 <\\s>', '<s> the chest shows no chnage of opacity at right lower lung <\\s>', '<s> the remaining lungs and left costophrenic angles are clear <\\s>', '<s> the heart is mild enlarged <\\s>', '<s> the bony thorax is intact <\\s

# Edema